# Comprehensive Analysis of Greenland Glacier-Ocean Interactions: Summer (June–July–August, JJA) Dynamics (2010–2020)

This computational notebook presents an integrated computational framework to investigate the dynamic interplay between Greenland’s marine-terminating glaciers and adjacent oceanographic conditions, focusing on four critical outlet glaciers (Helheim, Storebjørn, Daugaard-Jensen, and Waltershausen) spanning the 69°N latitudinal divide (Seale et al., 2011). Leveraging multi-source datasets including ORAS5 ocean reanalysis (CDS, 2021), IBCAO Version 5.0 bathymetry (Jakobsson et al., 2024), and glacier hydrographic records (Karlsson et al., 2023), the workflow systematically addresses polar data challenges through three interconnected analytical pipelines: (1) ocean temperature extraction at 300m depth using Euclidean distance-based nearest grid point search with progressive spatial search (1–3° radii) (Section 1), (2) high-resolution geospatial visualization integrating Cartopy, Rasterio, and Google Terrain tiles (Section 2), and (3) time-series analysis of summer (JJA) glacier dynamics—runoff, ice discharge, and submarine melt rates—with NE/SE regional partitioning (Sections 3–4). The 300m-depth ocean potential temperature visualization (Section 4) presents Section 1's extracted data as regional time series, while Section 5 synthesizes these components by computing submarine melt rates using the Slater & Straneo (2022) parameterization (ṁ = 0.142 × Q⁰·³¹ × TF¹·¹⁹, where Q is subglacial discharge in m³/s and TF is thermal forcing in °C), followed by statistical correlation testing between estimated melt rates and observed ice discharge. Technical innovations include automated coordinate system validation, multi-level data gap handling, and publication-ready 300DPI outputs with standardized metadata tags. By integrating multi-scale datasets within a reproducible computational framework, this study advances our capacity to quantify glacier-ocean interactions while establishing a template for future high-latitude cryosphere-hydrosphere studies.

In [1]:
pwd

'/Users/kyle/Documents/dissertation'

In [2]:
!mamba install -c conda-forge netcdf4 -y

warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/
[+] 0.0s
[+] 0.0s
[+] 0.1s
conda-forge/noarch    ━━━━━━━━━━━━━━╸━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
conda-forge/osx-arm64 ━╸━━━━━━━━━━━━━━━╸━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
pkgs/main/noarch      ━━━━━━━━━━━━━━━╸━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
pkgs/main/osx-arm64   ━━━━━━━━━━━╸━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/r/noarch         ╸━━━━━━━━━━━━━━━╸━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s[+] 0.2s
conda-forge/noarch    ╸━━━━━━━━━━━━━━━╸━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
conda-forge/osx-arm64 ━━━╸━━━━━━━━━━━━━━━╸━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
pkgs/main/osx-arm64   ━━━━━━━━━━━━━━━━━━━━━━  75.8kB /   5.9MB @ 493.2kB/s  0.2s
pkgs/r/osx-arm64      ━━━━━━━╸━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/

## Section 1: Ocean Temperature Extraction Pipeline from ORAS5 for Greenland Glaciers

This pipeline extracts monthly ocean temperature data at 300m depth from ORAS5 reanalysis (CDS, 2021) for four Greenland outlet glaciers spanning 2010-2020. I focus on 300m depth because in-situ observations from Sermilik Fjord show that waters at this depth maintain consistently warm temperatures (~4°C) year-round with minimal seasonal variability, in contrast to the highly variable surface layer (Straneo et al., 2010). This depth corresponds to the zone where submarine melt rates typically maximize (Seale et al., 2011) and captures the core of intruding Atlantic Water masses in southeast Greenland fjords. Seale et al. (2011) demonstrated that the retreat of East Greenland glaciers south of 69°N was strongly influenced by warming of subsurface coastal waters (100m to bottom), emphasizing the importance of deep-layer thermal forcing rather than surface conditions in driving glacier dynamics.

The methodology employs a two-stage approach. First, we identified the nearest valid ocean grid point with 300m depth data for each glacier using a progressive spatial search algorithm (1-3° radii) centered on original glacier coordinates. This yielded final ocean points at distances ranging from 0 km (Daugaard-Jensen) to 137 km (Waltershausen), with validation temperatures showing expected thermal dichotomy: SE glaciers (Helheim 4.33°C, Storebjoern 4.60°C) versus NE glaciers (Daugaard-Jensen 0.06°C, Waltershausen -0.04°C) in July 2015. Second, we systematically processed 33 monthly NetCDF files filtering for summer months (June–July–August, JJA) and extracting temperatures at each glacier's finalized ocean point using Euclidean distance minimization on the ORAS5 curvilinear grid, yielding 132 data points (4 glaciers × 3 months × 11 years). Regional averages were calculated as SE = mean(Helheim, Storebjoern) and NE = mean(Daugaard-Jensen, Waltershausen), producing an 11-year time series with SE regional mean of 4.75°C and NE regional mean of 0.06°C—a 4.70°C temperature difference that clearly delineates the 69°N oceanographic boundary identified by Seale et al. (2011).

The large spatial offsets between glacier termini and ocean points reflect ORAS5's ~25km resolution relative to narrow fjord geometries but remain physically justified as they capture regional water mass properties (Atlantic versus Polar Water) rather than localized fjord circulation. All extracted data is stored in glacier_temperature_300m_2010_2020.csv for subsequent time series visualization and correlation analysis with glacier discharge patterns.

In [3]:
# =============================================================================
# Section 1, Code 1: Ocean Data Point Identification for ORAS5 Extraction
# =============================================================================
# This script identifies the nearest valid ocean grid point with 300m-depth
# data for each of four Greenland outlet glaciers. Because ORAS5 (~25 km
# resolution) does not resolve narrow fjord geometries, glacier terminus
# coordinates often fall on land-masked grid cells. A progressive spatial
# search (1°, 2°, 3° radii) centred on each glacier locates the closest
# ocean cell that contains valid temperature data at the target depth.
#
# The resulting "replacement coordinates" are used in all subsequent
# temperature extraction steps (Section 1 Code 2, Section 4, Section 5).
#
# Data source: ORAS5 ocean reanalysis (CDS, 2021)
# Target depth: 300m — captures intruding Atlantic Water and minimises
#               seasonal surface variability (Straneo et al., 2010)
# =============================================================================

import xarray as xr
import numpy as np
import os
import pandas as pd
from geopy.distance import geodesic


def find_replacement_coordinates():
    """
    Identify the nearest valid ORAS5 ocean grid point at 300m depth
    for each glacier terminus.

    Method:
        1. Open a sample ORAS5 NetCDF file (July 2015).
        2. Find the depth index closest to 300m.
        3. For each glacier, progressively expand a spatial search
           window (1°→2°→3°) until a non-NaN ocean cell is found.
        4. Among valid cells, select the one with the smallest
           geodesic distance to the original glacier coordinates.

    Returns:
        dict: Replacement coordinates for each glacier, including
              latitude, longitude, distance from original, and
              a sample temperature value for validation.
    """
    base_dir = "/Users/kyle/Documents/dissertation/ORAS5/votemper"

    # Original glacier terminus coordinates (Source: Google Earth, 2025)
    original_glacier_locations = {
        "Helheim Gletsjer":         {"lat": 66.3500, "lon": -38.2000},   # 66°21'00"N, 38°12'00"W
        "Storebjoern":              {"lat": 63.7167, "lon": -41.6497},   # 63°43'00"N, 41°38'59"W
        "Daugaard-Jensen Gletsjer": {"lat": 71.8706, "lon": -28.7011},   # 71°52'14"N, 28°42'04"W
        "Waltershausen Gletsjer":   {"lat": 73.9000, "lon": -24.4167},   # 73°54'00"N, 24°25'00"W
    }

    print("=== FINDING REPLACEMENT COORDINATES WITH 300m DATA ===")

    # Use July 2015 (OPER) as the sample file for grid inspection
    sample_file = "votemper_control_monthly_highres_3D_201507_OPER_v0.1.nc"
    file_path = os.path.join(base_dir, sample_file)

    if not os.path.exists(file_path):
        print(f"  Sample file not found: {sample_file}")
        return original_glacier_locations

    replacement_coordinates = {}

    with xr.open_dataset(file_path) as ds:
        # Find depth index closest to 300m
        depth_idx = np.abs(ds.deptht.values - 300).argmin()
        actual_depth = ds.deptht.values[depth_idx]
        print(f"Target depth: 300m, Closest available: {actual_depth:.1f}m\n")

        for glacier, original_pos in original_glacier_locations.items():
            print(f"Processing {glacier}...")
            original_lat = original_pos['lat']
            original_lon = original_pos['lon']
            print(f"  Original: {original_lat:.4f}°N, {original_lon:.4f}°E")

            # Progressive spatial search with expanding radii
            search_ranges = [1.0, 2.0, 3.0, 4.0, 5.0]
            found_point = None

            for search_range in search_ranges:
                search_lat_min = original_lat - search_range
                search_lat_max = original_lat + search_range
                search_lon_min = original_lon - search_range
                search_lon_max = original_lon + search_range

                # Create spatial mask within search window
                region_mask = (
                    (ds.nav_lat >= search_lat_min) & (ds.nav_lat <= search_lat_max) &
                    (ds.nav_lon >= search_lon_min) & (ds.nav_lon <= search_lon_max)
                )

                # Create data availability mask (non-NaN at 300m depth)
                data_mask = ~np.isnan(ds.votemper.isel(time_counter=0, deptht=depth_idx))
                valid_mask = region_mask & data_mask

                if valid_mask.any():
                    # Collect all valid ocean points in the search window
                    valid_points = []
                    valid_indices = np.where(valid_mask.values)

                    for i in range(len(valid_indices[0])):
                        y, x = valid_indices[0][i], valid_indices[1][i]
                        lat = ds.nav_lat.values[y, x]
                        lon = ds.nav_lon.values[y, x]
                        temp = ds.votemper.isel(
                            time_counter=0, deptht=depth_idx, y=y, x=x
                        ).values.item()
                        valid_points.append((lat, lon, temp))

                    # Select the point closest to the original glacier position
                    best_point = min(
                        valid_points,
                        key=lambda p: geodesic(
                            (original_lat, original_lon), (p[0], p[1])
                        ).km
                    )

                    best_lat, best_lon, best_temp = best_point
                    distance = geodesic(
                        (original_lat, original_lon), (best_lat, best_lon)
                    ).km

                    print(f"  Found at {search_range}° range: "
                          f"{best_lat:.4f}°N, {best_lon:.4f}°E")
                    print(f"     Temperature: {best_temp:.2f}°C, "
                          f"Distance: {distance:.2f} km")

                    found_point = {
                        'lat': best_lat,
                        'lon': best_lon,
                        'original_distance_km': distance,
                        'temperature_sample': best_temp
                    }
                    break  # Stop expanding search radius once a point is found

            if found_point:
                replacement_coordinates[glacier] = found_point
            else:
                print(f"  No 300m data found within 3° — using original coordinates")
                replacement_coordinates[glacier] = original_pos

    return replacement_coordinates


# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    replacement_locations = find_replacement_coordinates()

    print("\n" + "=" * 60)
    print("FINAL REPLACEMENT COORDINATES:")
    print("=" * 60)
    for glacier, coords in replacement_locations.items():
        if 'temperature_sample' in coords:
            print(f"  {glacier}: "
                  f"{{'lat': {coords['lat']:.4f}, 'lon': {coords['lon']:.4f}}}  "
                  f"# Temp: {coords['temperature_sample']:.2f}°C, "
                  f"Dist: {coords['original_distance_km']:.1f} km")
        else:
            print(f"  {glacier}: "
                  f"{{'lat': {coords['lat']:.4f}, 'lon': {coords['lon']:.4f}}}  "
                  f"# Using original (no 300m data found)")

=== FINDING REPLACEMENT COORDINATES WITH 300m DATA ===
Target depth: 300m, Closest available: 300.9m

Processing Helheim Gletsjer...
  Original: 66.3500°N, -38.2000°E
  Found at 2.0° range: 65.2358°N, -37.9326°E
     Temperature: 4.33°C, Distance: 124.84 km
Processing Storebjoern...
  Original: 63.7167°N, -41.6497°E
  Found at 1.0° range: 62.9264°N, -40.8969°E
     Temperature: 4.66°C, Distance: 95.83 km
Processing Daugaard-Jensen Gletsjer...
  Original: 71.8706°N, -28.7011°E
  Found at 4.0° range: 71.1908°N, -25.1324°E
     Temperature: 0.01°C, Distance: 147.24 km
Processing Waltershausen Gletsjer...
  Original: 73.9000°N, -24.4167°E
  Found at 3.0° range: 72.0131°N, -22.0269°E
     Temperature: -0.04°C, Distance: 224.58 km

FINAL REPLACEMENT COORDINATES:
  Helheim Gletsjer: {'lat': 65.2358, 'lon': -37.9326}  # Temp: 4.33°C, Dist: 124.8 km
  Storebjoern: {'lat': 62.9264, 'lon': -40.8969}  # Temp: 4.66°C, Dist: 95.8 km
  Daugaard-Jensen Gletsjer: {'lat': 71.1908, 'lon': -25.1324}  # Te

In [4]:
# =============================================================================
# Section 1, Code 2: ORAS5 Ocean Temperature Extraction Pipeline
# =============================================================================
# This script extracts monthly ocean potential temperature at 300m depth
# from ORAS5 reanalysis (CDS, 2021) for four Greenland outlet glaciers
# over the period 2010–2020, filtering for summer months (June–July–August,
# JJA).
#
# The extraction uses "replacement coordinates" identified in Code 1,
# which represent the nearest valid ocean grid points to each glacier
# terminus. The pipeline processes 33 monthly NetCDF files (3 JJA months
# × 11 years) and produces 132 data points (4 glaciers × 33 months).
#
# Grid point matching uses Euclidean distance minimisation on the ORAS5
# curvilinear grid (nav_lat, nav_lon), consistent with the approach in
# Sections 4 and 5.
#
# Output: glacier_temperature_300m_2010_2020.csv
# =============================================================================

import xarray as xr
import numpy as np
import os
import pandas as pd
from geopy.distance import geodesic

# Configuration
base_dir = "/Users/kyle/Documents/dissertation/ORAS5/votemper"
output_folder = "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots"
os.makedirs(output_folder, exist_ok=True)

# Final replacement coordinates from Code 1 (nearest valid ocean points at 300m)
replacement_locations = {
    "Helheim Gletsjer":         {"lat": 65.2358, "lon": -37.9326},   
    "Storebjoern":              {"lat": 62.9264, "lon": -40.8969},   
    "Daugaard-Jensen Gletsjer": {"lat": 71.1908, "lon": -25.1324},   
    "Waltershausen Gletsjer":   {"lat": 72.0131, "lon": -22.0269}    
}

# Regional classification divided at 69°N (Seale et al., 2011)
region_tags = {
    "Helheim Gletsjer": "SE",
    "Storebjoern": "SE",
    "Daugaard-Jensen Gletsjer": "NE",
    "Waltershausen Gletsjer": "NE"
}


def extract_month_from_filename(filename):
    """
    Extract year and month from an ORAS5 NetCDF filename.

    Expected pattern: votemper_control_monthly_highres_3D_YYYYMM_*.nc
    The 6th underscore-delimited token contains the YYYYMM string.

    Args:
        filename (str): NetCDF filename.

    Returns:
        tuple: (year, month) as integers.

    Raises:
        ValueError: If YYYYMM cannot be parsed from the filename.
    """
    try:
        # Primary method: split by underscore, 6th element is YYYYMM
        year_month = filename.split('_')[5]
        month = int(year_month[4:6])
        year = int(year_month[:4])
        return year, month
    except (IndexError, ValueError):
        # Fallback: regex search for 6-digit date string
        import re
        match = re.search(r'_(\d{6})_', filename)
        if match:
            year_month = match.group(1)
            return int(year_month[:4]), int(year_month[4:6])
        else:
            raise ValueError(f"Cannot extract date from filename: {filename}")


def quick_test_with_coordinates(coordinates_dict):
    """
    Quick validation test using three summer 2015 files.

    Extracts 300m-depth temperature for each glacier from June, July,
    and August 2015 to verify that the replacement coordinates yield
    valid (non-NaN) data at the expected depth.

    Args:
        coordinates_dict (dict): Glacier coordinates to test.

    Returns:
        pd.DataFrame: Extracted temperature data for validation.
    """
    print("\n=== QUICK VALIDATION TEST (Summer 2015) ===")

    test_files = [
        "votemper_control_monthly_highres_3D_201506_OPER_v0.1.nc",
        "votemper_control_monthly_highres_3D_201507_OPER_v0.1.nc",
        "votemper_control_monthly_highres_3D_201508_OPER_v0.1.nc",
    ]

    all_data = []

    for i, file in enumerate(test_files):
        print(f"\n  Testing file {i+1}/3: {file}")
        file_path = os.path.join(base_dir, file)

        if not os.path.exists(file_path):
            print(f"    File not found: {file}")
            continue

        try:
            with xr.open_dataset(file_path) as ds:
                # Find depth index closest to 300m
                depth_idx = np.abs(ds.deptht.values - 300).argmin()
                actual_depth = ds.deptht.values[depth_idx]
                print(f"    Depth: {actual_depth:.1f}m")

                # Extract year and month from filename
                year, month = extract_month_from_filename(file)
                print(f"    Date: {year}-{month:02d}")

                for glacier, pos in coordinates_dict.items():
                    lat, lon = pos['lat'], pos['lon']

                    # Find nearest grid point via Euclidean distance
                    dist_matrix = (ds.nav_lon - lon)**2 + (ds.nav_lat - lat)**2
                    y_idx, x_idx = np.unravel_index(
                        np.argmin(dist_matrix.values), dist_matrix.shape
                    )

                    # Extract temperature at 300m
                    temp = ds.votemper.isel(
                        time_counter=0, deptht=depth_idx, y=y_idx, x=x_idx
                    ).values.item()

                    status = "OK" if not np.isnan(temp) else "NaN"
                    print(f"    [{status}] {glacier}: {temp:.2f}°C")

                    if not np.isnan(temp):
                        all_data.append({
                            'Glacier': glacier,
                            'Year': year,
                            'Month': month,
                            'OceanTemp': temp,
                            'Region': region_tags[glacier]
                        })

        except Exception as e:
            print(f"    Error: {e}")

    return pd.DataFrame(all_data)


def process_full_dataset(coordinates_dict, start_year=2010, end_year=2020):
    """
    Extract summer (JJA) ocean temperature at 300m depth for all glaciers
    across the full study period (2010–2020).

    Iterates over all NetCDF files in the data directory, filters for
    June–July–August within the specified year range, and extracts
    temperature at each glacier's replacement ocean grid point.

    Args:
        coordinates_dict (dict): Glacier replacement coordinates.
        start_year (int): First year of the study period.
        end_year (int): Last year of the study period.

    Returns:
        pd.DataFrame: Columns — Glacier, Year, Month, OceanTemp, Region.
    """
    print(f"\n=== PROCESSING FULL DATASET {start_year}–{end_year} (JJA only) ===")

    all_data = []
    processed_files = 0

    # List all NetCDF files in the data directory
    all_files = [f for f in os.listdir(base_dir) if f.endswith('.nc')]
    print(f"  Found {len(all_files)} NetCDF files in directory")

    for file in all_files:
        try:
            # Extract date from filename
            year, month = extract_month_from_filename(file)

            # Filter: summer months (JJA) within the study period only
            if month not in [6, 7, 8] or year < start_year or year > end_year:
                continue

            file_path = os.path.join(base_dir, file)

            with xr.open_dataset(file_path) as ds:
                # Find depth index closest to 300m
                depth_idx = np.abs(ds.deptht.values - 300).argmin()

                for glacier, pos in coordinates_dict.items():
                    lat, lon = pos['lat'], pos['lon']

                    # Find nearest grid point via Euclidean distance
                    dist_matrix = (ds.nav_lon - lon)**2 + (ds.nav_lat - lat)**2
                    y_idx, x_idx = np.unravel_index(
                        np.argmin(dist_matrix.values), dist_matrix.shape
                    )

                    # Extract temperature at 300m
                    temp = ds.votemper.isel(
                        time_counter=0, deptht=depth_idx, y=y_idx, x=x_idx
                    ).values.item()

                    if not np.isnan(temp):
                        all_data.append({
                            'Glacier': glacier,
                            'Year': year,
                            'Month': month,
                            'OceanTemp': temp,
                            'Region': region_tags[glacier]
                        })

                processed_files += 1
                if processed_files % 10 == 0:
                    print(f"    Processed {processed_files} files...")

        except Exception as e:
            print(f"    Error with {file}: {e}")
            continue

    print(f"  Completed: {processed_files} files processed, "
          f"{len(all_data)} data points collected")
    return pd.DataFrame(all_data)


# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    print("=" * 60)
    print("ORAS5 TEMPERATURE EXTRACTION PIPELINE")
    print("=" * 60)
    print("\nUsing replacement coordinates (from Code 1):")
    for glacier, coords in replacement_locations.items():
        print(f"  {glacier}: {coords['lat']:.4f}°N, {coords['lon']:.4f}°E")

    # Step 1: Quick validation test with summer 2015 data
    test_results = quick_test_with_coordinates(replacement_locations)

    print("\n" + "=" * 60)
    print("QUICK TEST RESULTS:")
    if not test_results.empty:
        print(test_results.to_string(index=False))

        # Summer 2015 averages per glacier
        summer_avg = test_results.groupby(['Glacier', 'Region'])['OceanTemp'].mean()
        print(f"\nSummer (JJA) 2015 Averages:")
        for (glacier, region), temp in summer_avg.items():
            print(f"  {glacier} ({region}): {temp:.2f}°C")
    else:
        print("  No data collected — check file paths and coordinates")

    # Step 2: Process full 2010–2020 dataset
    print("\n" + "=" * 60)
    print("PROCESSING FULL 2010–2020 DATASET...")
    full_results = process_full_dataset(replacement_locations)

    if not full_results.empty:
        print(f"\nFULL DATASET RESULTS ({len(full_results)} data points):")

        # Yearly summer (JJA) averages
        yearly_avg = full_results.groupby(
            ['Glacier', 'Year']
        )['OceanTemp'].mean().reset_index()
        print("\nYearly Summer (JJA) Averages:")
        print(yearly_avg.to_string(index=False))

        # Regional summary statistics
        for region in ['SE', 'NE']:
            region_temps = full_results[full_results['Region'] == region]['OceanTemp']
            print(f"\n{region} Region: mean = {region_temps.mean():.2f}°C, "
                  f"min = {region_temps.min():.2f}°C, "
                  f"max = {region_temps.max():.2f}°C")

        # Save to CSV
        csv_path = os.path.join(output_folder, "glacier_temperature_300m_2010_2020.csv")
        full_results.to_csv(csv_path, index=False)
        print(f"\nData saved to: {csv_path}")
    else:
        print("  No data collected — check file paths and coordinates")

ORAS5 TEMPERATURE EXTRACTION PIPELINE

Using replacement coordinates (from Code 1):
  Helheim Gletsjer: 65.2358°N, -37.9326°E
  Storebjoern: 62.9264°N, -40.8969°E
  Daugaard-Jensen Gletsjer: 71.1908°N, -25.1324°E
  Waltershausen Gletsjer: 72.0131°N, -22.0269°E

=== QUICK VALIDATION TEST (Summer 2015) ===

  Testing file 1/3: votemper_control_monthly_highres_3D_201506_OPER_v0.1.nc
    Depth: 300.9m
    Date: 2015-06
    [OK] Helheim Gletsjer: 3.93°C
    [OK] Storebjoern: 4.23°C
    [OK] Daugaard-Jensen Gletsjer: -0.03°C
    [OK] Waltershausen Gletsjer: 0.02°C

  Testing file 2/3: votemper_control_monthly_highres_3D_201507_OPER_v0.1.nc
    Depth: 300.9m
    Date: 2015-07
    [OK] Helheim Gletsjer: 4.33°C
    [OK] Storebjoern: 4.66°C
    [OK] Daugaard-Jensen Gletsjer: 0.01°C
    [OK] Waltershausen Gletsjer: -0.04°C

  Testing file 3/3: votemper_control_monthly_highres_3D_201508_OPER_v0.1.nc
    Depth: 300.9m
    Date: 2015-08
    [OK] Helheim Gletsjer: 4.83°C
    [OK] Storebjoern: 4.80°C


## Section 2: Advanced Glacier-Ocean Visualization System with Multi-Source Geospatial Integration

This sophisticated geospatial visualization system integrates Cartopy (v0.24.1) for orthographic projections and map elements, Rasterio (v1.4.3) for processing IBCAO Version 5.0 bathymetric data (Jakobsson et al., 2024) at native 100m resolution (2024 release), and Geopy (v2.4.1) for precise geodesic distance calculations between glacier termini and ocean sampling locations. The implementation creates a comprehensive visualization framework that combines high-resolution bathymetric data (100m resolution GeoTIFF) with satellite imagery through a carefully designed dual-map approach, incorporating fallback synthetic bathymetry generation for missing data scenarios. The orthographic overview map utilizes Cartopy's advanced coordinate reference system (CRS) transformations to display glacier-ocean pairs within their regional context, while detailed maps employ Google Terrain tiles (Google Maps API, 2025) through Cartopy's image tile interface for localized examination.

The system implements a rigorous coordinate transformation pipeline using Rasterio's warp functionality to convert between WGS84 (EPSG:4326) and IBCAO's native CRS (EPSG:3996), ensuring accurate positioning of bathymetric data. Geopy's geodesic function calculates great-circle distances between glacier termini (marked with 14pt red circles) and their corresponding ocean points (12pt blue X markers), with connection lines (1.8pt purple dashes) dynamically labeled using Cartopy's advanced text positioning system. The bathymetric visualization employs a specialized processing chain that includes data windowing, coordinate transformation, and depth-based masking (-500m to 0m range) to optimize both performance and visual clarity.

For regional context, the visualization incorporates multiple Cartopy features including 50m-resolution coastline vectors, land masks with custom coloring (#f0e6dc), and a 69°N reference line marking the NE/SE boundary as established in Seale et al. (2011). The technical implementation demonstrates tight integration between these libraries - Rasterio handles the bathymetric data extraction and CRS transformations, Cartopy manages the map projections and visual elements, while Geopy provides the essential spatial calculations that connect the terrestrial and marine components of the study. Outputs are generated as publication-ready 300DPI images with print-optimized color schemes, maintaining consistent styling across both overview (14×12") and detailed (12×10") maps through shared color schemes and annotation styles. All maps contextualize the summer (JJA) study period by displaying glacier-ocean point pairs used in subsequent seasonal analyses.

In [5]:
!pip install cartopy

In [6]:
import cartopy
print(cartopy.__version__)

0.24.1


In [7]:
!pip install geopy

In [8]:
import geopy
print(geopy.__version__)

2.4.1


In [9]:
pip install rasterio

Note: you may need to restart the kernel to use updated packages.


In [10]:
import rasterio
print(rasterio.__version__)

1.4.3


In [11]:
# =============================================================================
# Section 2: Geospatial Visualisation — Greenland Overview and Glacier Maps
# =============================================================================
# This script generates two types of maps:
#
#   (1) Greenland Overview Map — Orthographic projection centred on the study
#       area, with IBCAO v5.0 bathymetry (Jakobsson et al., 2024) clipped to
#       0–500m depth. Original glacier terminus locations (red circles) and
#       replacement ocean data points (blue crosses) are connected by dashed
#       lines annotated with geodesic distances (km). A 69°N reference line
#       divides the NE/SE regions (Seale et al., 2011).
#
#   (2) Individual Glacier Detail Maps — Satellite imagery (Google Tiles) for
#       each glacier, showing original vs. replacement coordinates with DMS
#       annotations and distance labels.
#
# Dependencies: cartopy (map projections), rasterio (IBCAO GeoTIFF),
#               geopy (geodesic distances)
#
# Data sources:
#   - IBCAO v5.0 bathymetry: Jakobsson et al. (2024), 100m resolution
#   - Glacier coordinates: Google Earth (2025)
#   - Satellite tiles: Google Maps API (2025)
# =============================================================================

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
from matplotlib.lines import Line2D
import os
import numpy as np
from geopy.distance import geodesic
import rasterio
from rasterio.warp import transform as rio_transform


# =====================================================================
# Glacier Coordinate Definitions
# =====================================================================

# Original glacier terminus locations (Source: Google Earth, 2025)
original_locations = {
    "Helheim Gletsjer": {
        "lat": 66 + 21/60,               # 66°21'00"N
        "lon": -(38 + 12/60),             # 38°12'00"W
        "region": "SE"
    },
    "Storebjoern": {
        "lat": 63 + 43/60,               # 63°43'00"N
        "lon": -(41 + 38/60 + 59/3600),   # 41°38'59"W
        "region": "SE"
    },
    "Daugaard-Jensen Gletsjer": {
        "lat": 71 + 52/60 + 14/3600,      # 71°52'14"N
        "lon": -(28 + 42/60 + 4/3600),    # 28°42'04"W
        "region": "NE"
    },
    "Waltershausen Gletsjer": {
        "lat": 73 + 54/60,               # 73°54'00"N
        "lon": -(24 + 25/60),             # 24°25'00"W
        "region": "NE"
    }
}

# Replacement coordinates: nearest valid ORAS5 ocean grid points at 300m
# depth, identified by Section 1 Code 1
replacement_locations = {
    "Helheim Gletsjer":         {"lat": 65.2358, "lon": -37.9326},   
    "Storebjoern":              {"lat": 62.9264, "lon": -40.8969},   
    "Daugaard-Jensen Gletsjer": {"lat": 71.1908, "lon": -25.1324},   
    "Waltershausen Gletsjer":   {"lat": 72.0131, "lon": -22.0269}    
}

# Output directory
output_folder = "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots"
os.makedirs(output_folder, exist_ok=True)


# =====================================================================
# Utility Functions
# =====================================================================

def decimal_to_dms(coord, is_lat):
    """
    Convert decimal degree coordinate to degrees-minutes-seconds string.

    Args:
        coord (float): Decimal degree value.
        is_lat (bool): True for latitude (N/S), False for longitude (E/W).

    Returns:
        str: Formatted DMS string, e.g. '66°21'0.0"N'.
    """
    abs_coord = abs(coord)
    degrees = int(abs_coord)
    remaining = (abs_coord - degrees) * 60
    minutes = int(remaining)
    seconds = round((remaining - minutes) * 60, 1)

    # Handle rounding overflow
    if seconds >= 60:
        seconds = 0
        minutes += 1
    if minutes >= 60:
        minutes = 0
        degrees += 1

    if is_lat:
        direction = "N" if coord >= 0 else "S"
    else:
        direction = "E" if coord >= 0 else "W"

    return f"{degrees}°{minutes}'{seconds}\"{direction}"


# =====================================================================
# Figure 1: Greenland Overview Map with IBCAO Bathymetry
# =====================================================================

def create_greenland_map():
    """
    Create a Greenland overview map showing original glacier locations,
    replacement ocean data points, and IBCAO v5.0 bathymetry (0–500m).

    The map uses an Orthographic projection centred on the study area
    (-40°E, 72°N). A 69°N reference line marks the NE/SE regional
    boundary (Seale et al., 2011). Connection lines between original
    and replacement coordinates are annotated with geodesic distances.

    Returns:
        str: File path of the saved figure.
    """
    fig = plt.figure(figsize=(14, 12))
    proj = ccrs.Orthographic(central_longitude=-40, central_latitude=72)
    ax = fig.add_subplot(1, 1, 1, projection=proj)

    # Map display extent (visible area)
    display_extent = [-45, -25, 62, 74.5]
    ax.set_extent(display_extent, crs=ccrs.PlateCarree())

    # Data loading extent (wider than display to avoid edge artefacts)
    data_extent = [-43, -5, 50, 75]

    bathymetry_path = "/Users/kyle/Documents/dissertation/ibcao-5.0/ibcao-2024-100m-20240701-depth.tiff"

    try:
        with rasterio.open(bathymetry_path) as src:
            print("=== Bathymetry Data Loading ===")
            print(f"  CRS: {src.crs}")

            # Coordinate transformation: WGS84 → IBCAO CRS (EPSG:3996)
            src_crs = 'EPSG:4326'
            dst_crs = src.crs

            xmin, ymin = rio_transform(
                src_crs, dst_crs,
                [data_extent[0]], [data_extent[2]]
            )
            xmax, ymax = rio_transform(
                src_crs, dst_crs,
                [data_extent[1]], [data_extent[3]]
            )

            # Calculate pixel window from projected coordinates
            transform = src.transform
            inv_transform = ~transform

            left, bottom = inv_transform * (xmin[0], ymin[0])
            right, top = inv_transform * (xmax[0], ymax[0])

            col_start = max(0, int(left))
            row_start = max(0, int(top))
            col_stop = min(src.width, int(right))
            row_stop = min(src.height, int(bottom))

            width = col_stop - col_start
            height = row_stop - row_start

            # Read bathymetry data within window
            window = rasterio.windows.Window(
                col_start, row_start, width, height)
            bathymetry = src.read(1, window=window)
            window_transform = src.window_transform(window)

            # Build coordinate grids in IBCAO CRS
            rows, cols = bathymetry.shape
            x_coords = np.linspace(
                window_transform[2],
                window_transform[2] + window_transform[0] * cols,
                cols
            )
            y_coords = np.linspace(
                window_transform[5],
                window_transform[5] + window_transform[4] * rows,
                rows
            )
            xx, yy = np.meshgrid(x_coords, y_coords)

            # Transform IBCAO CRS → WGS84 for plotting
            lon, lat = rio_transform(
                dst_crs, src_crs,
                np.array(xx).flatten(),
                np.array(yy).flatten()
            )
            lon = np.array(lon).reshape(xx.shape)
            lat = np.array(lat).reshape(yy.shape)

            # Mask land (>=0) and values outside data extent
            bathymetry = np.ma.masked_where(
                (bathymetry >= 0) |
                (lon < data_extent[0]) | (lon > data_extent[1]) |
                (lat < data_extent[2]) | (lat > data_extent[3]),
                bathymetry
            )
            # Clip to 0–500m depth range (relevant for glacier-ocean interaction)
            bathymetry = np.clip(bathymetry, -500, 0)

    except Exception as e:
        print(f"  Bathymetry loading failed: {e}")
        print("  Using synthetic fallback data")
        lon = np.linspace(data_extent[0], data_extent[1], 1000)
        lat = np.linspace(data_extent[2], data_extent[3], 1000)
        lon, lat = np.meshgrid(lon, lat)
        bathymetry = -1000 * np.exp(-((lon + 35)**2 + (lat - 68)**2) / 50)
        bathymetry = np.clip(bathymetry, -1000, 0)

    # --- Bathymetry contour fill ---
    levels = np.linspace(-500, 0, 11)
    cs = ax.contourf(lon, lat, bathymetry, levels=levels,
                     cmap='Blues_r', extend='both',
                     transform=ccrs.PlateCarree(), alpha=0.8)

    # --- Base map features ---
    ax.add_feature(cfeature.LAND, facecolor='#f0e6dc', zorder=2)
    ax.add_feature(cfeature.COASTLINE, edgecolor='#5c5c5c', zorder=3)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=1.2, alpha=0.7)
    ax.gridlines(draw_labels=True, linewidth=0.7, color='gray',
                 alpha=0.6, linestyle='--')
    ax.add_feature(cfeature.LAND.with_scale('50m'),
                   facecolor='#f0e6dc', edgecolor='none', zorder=1)

    # Colour bar
    cbar = plt.colorbar(cs, ax=ax, shrink=0.6)
    cbar.set_label('Ocean Depth (m)', fontsize=12)

    # --- Glacier markers ---
    # Original locations (red circles)
    for glacier, loc in original_locations.items():
        ax.plot(loc["lon"], loc["lat"],
                marker='o', markersize=14, color='red',
                markeredgecolor='black',
                transform=ccrs.PlateCarree())

    # Replacement locations (blue crosses)
    for glacier, loc in replacement_locations.items():
        ax.plot(loc["lon"], loc["lat"],
                marker='X', markersize=12, color='blue',
                markeredgecolor='black',
                transform=ccrs.PlateCarree())

    # --- Connection lines with distance labels ---
    for glacier in original_locations:
        orig = original_locations[glacier]
        repl = replacement_locations[glacier]

        ax.plot([orig["lon"], repl["lon"]], [orig["lat"], repl["lat"]],
                color='purple', linestyle='--', linewidth=1.8, alpha=0.8,
                transform=ccrs.PlateCarree())

        # Geodesic distance
        distance = geodesic(
            (orig["lat"], orig["lon"]),
            (repl["lat"], repl["lon"])
        ).km
        mid_lon = (orig["lon"] + repl["lon"]) / 2
        mid_lat = (orig["lat"] + repl["lat"]) / 2

        # Per-glacier label positioning to avoid overlap
        if glacier == "Waltershausen Gletsjer":
            text_transform = offset_copy(
                ccrs.Geodetic()._as_mpl_transform(ax),
                units='dots', x=-40, y=-60)
            ha = 'right'
        elif glacier == "Daugaard-Jensen Gletsjer":
            text_transform = offset_copy(
                ccrs.Geodetic()._as_mpl_transform(ax),
                units='dots', x=-80, y=-40)
            ha = 'right'
        else:
            text_transform = offset_copy(
                ccrs.Geodetic()._as_mpl_transform(ax),
                units='dots', x=50, y=0)
            ha = 'left'

        ax.text(mid_lon, mid_lat, f"{distance:.1f} km",
                verticalalignment='center',
                horizontalalignment=ha,
                transform=text_transform,
                bbox=dict(facecolor='white', alpha=0.8, boxstyle='round'),
                fontsize=10, weight='bold')

    # --- Glacier name labels ---
    for glacier, loc in original_locations.items():
        geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)

        if glacier in ("Waltershausen Gletsjer", "Daugaard-Jensen Gletsjer"):
            text_transform = offset_copy(
                geodetic_transform, units='dots', x=-60, y=0)
            ha = 'right'
        else:
            # Helheim and Storebjørn: label above marker
            text_transform = offset_copy(
                geodetic_transform, units='dots', x=0, y=80)
            ha = 'center'

        ax.text(loc["lon"], loc["lat"], glacier,
                verticalalignment='center',
                horizontalalignment=ha,
                transform=text_transform,
                bbox=dict(facecolor='white', alpha=0.8, boxstyle='round'),
                fontsize=11, weight='bold')

    # --- 69°N regional boundary line (Seale et al., 2011) ---
    lat_69 = 69.0
    lons = np.linspace(-75, -10, 100)
    lats = np.full_like(lons, lat_69)
    ax.plot(lons, lats, color='black', linestyle='--', linewidth=1.5,
            transform=ccrs.PlateCarree(), alpha=0.7)

    ax.text(-30, lat_69 - 0.5,
            "69°N Reference Latitude\n(Seale et al., 2011)",
            transform=ccrs.PlateCarree(),
            bbox=dict(facecolor='white', alpha=0.7, boxstyle='round'),
            fontsize=10, weight='bold')

    # NE/SE region labels
    label_params = {
        'transform': ccrs.PlateCarree(),
        'fontsize': 14,
        'weight': 'bold',
        'bbox': dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.3')
    }
    ax.text(-45, 70, "NORTHEAST (NE)", color='darkred', **label_params)
    ax.text(-45, 68, "SOUTHEAST (SE)", color='darkblue', **label_params)

    # --- Legend ---
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='red',
               markersize=12, markeredgecolor='black',
               label='Original Glacier Location'),
        Line2D([0], [0], marker='X', color='w', markerfacecolor='blue',
               markersize=12, markeredgecolor='black',
               label='Ocean Data Point (300m depth)'),
        Line2D([0], [0], color='purple', linestyle='--', lw=2,
               label='Connection')
    ]
    ax.legend(handles=legend_elements, loc='lower right', fontsize=10)

    plt.title("Greenland Glacier Locations with Ocean Data Points",
              fontsize=18, pad=25, weight='bold')

    save_path = os.path.join(output_folder, "greenland_overview_map.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    return save_path


# =====================================================================
# Figures 2–5: Individual Glacier Detail Maps (Satellite Imagery)
# =====================================================================

def create_glacier_detail_maps():
    """
    Create satellite-view detail maps for each glacier, showing the
    original terminus location vs. the replacement ocean data point.

    Each map includes:
    - Google satellite imagery at appropriate zoom level
    - Red circle (original) and blue cross (replacement) markers
    - Purple dashed connection line with distance annotation
    - DMS coordinate info box

    Returns:
        list: File paths of saved figures.
    """
    google_terrain = cimgt.GoogleTiles(style="satellite")
    saved_paths = []

    for glacier in original_locations:
        orig_loc = original_locations[glacier]
        repl_loc = replacement_locations[glacier]

        # Map extent: encompass both coordinates with padding
        all_lats = [orig_loc["lat"], repl_loc["lat"]]
        all_lons = [orig_loc["lon"], repl_loc["lon"]]

        lat_padding = max(0.8, 0.3 * abs(orig_loc["lat"] - repl_loc["lat"]))
        lon_padding = max(1.2, 0.3 * abs(orig_loc["lon"] - repl_loc["lon"]))

        extent = [
            min(all_lons) - lon_padding,
            max(all_lons) + lon_padding,
            min(all_lats) - lat_padding,
            max(all_lats) + lat_padding
        ]

        # Create map with satellite imagery
        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_subplot(1, 1, 1, projection=google_terrain.crs)
        ax.set_extent(extent, crs=ccrs.Geodetic())
        ax.add_image(google_terrain, 10)

        # Original location (red circle)
        ax.plot(orig_loc["lon"], orig_loc["lat"],
                marker='o', color='red', markersize=18,
                markeredgecolor='black', alpha=0.9,
                transform=ccrs.Geodetic())

        # Replacement location (blue cross)
        ax.plot(repl_loc["lon"], repl_loc["lat"],
                marker='X', color='blue', markersize=16,
                markeredgecolor='black', alpha=0.9,
                transform=ccrs.Geodetic())

        # Connection line
        ax.plot([orig_loc["lon"], repl_loc["lon"]],
                [orig_loc["lat"], repl_loc["lat"]],
                color='purple', linestyle='--', linewidth=2.2, alpha=0.9,
                transform=ccrs.Geodetic())

        # Distance label at midpoint
        distance = geodesic(
            (orig_loc["lat"], orig_loc["lon"]),
            (repl_loc["lat"], repl_loc["lon"])
        ).km
        mid_lon = (orig_loc["lon"] + repl_loc["lon"]) / 2
        mid_lat = (orig_loc["lat"] + repl_loc["lat"]) / 2

        ax.text(mid_lon, mid_lat, f"Distance: {distance:.1f} km",
                verticalalignment='center',
                horizontalalignment='center',
                transform=ccrs.Geodetic(),
                bbox=dict(facecolor='white', alpha=0.85, boxstyle='round'),
                fontsize=12, weight='bold')

        # DMS coordinate info box
        coord_info = (
            f"Original Location:\n"
            f"{decimal_to_dms(orig_loc['lat'], True)}\n"
            f"{decimal_to_dms(orig_loc['lon'], False)}\n\n"
            f"Ocean Data Point:\n"
            f"{decimal_to_dms(repl_loc['lat'], True)}\n"
            f"{decimal_to_dms(repl_loc['lon'], False)}"
        )

        ax.text(extent[1] - 0.1, extent[3] - 0.1, coord_info,
                verticalalignment='top',
                horizontalalignment='right',
                transform=ccrs.Geodetic(),
                bbox=dict(facecolor='white', alpha=0.85, boxstyle='round'),
                fontsize=10, family='monospace')

        # Legend
        legend_elements = [
            Line2D([0], [0], marker='o', color='w', markerfacecolor='red',
                   markersize=10, markeredgecolor='black',
                   label='Original Glacier Location'),
            Line2D([0], [0], marker='X', color='w', markerfacecolor='blue',
                   markersize=10, markeredgecolor='black',
                   label='Ocean Data Point (300m depth)')
        ]
        ax.legend(handles=legend_elements, loc='lower left', fontsize=10)

        plt.title(f"{glacier}: Glacier Location vs Ocean Data Point",
                  fontsize=16, pad=15, weight='bold')

        save_path = os.path.join(
            output_folder,
            f"{glacier.replace(' ', '_')}_Comparison.png"
        )
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        saved_paths.append(save_path)
        plt.close()

    return saved_paths


# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    print("=" * 60)
    print("GEOSPATIAL VISUALISATION PIPELINE")
    print("=" * 60)

    print("\nCreating Greenland Overview Map...")
    overview_path = create_greenland_map()
    print(f"  Greenland overview saved: {overview_path}")

    print("\nCreating Individual Glacier Detail Maps...")
    detail_paths = create_glacier_detail_maps()
    for path in detail_paths:
        print(f"  Detail map saved: {path}")

    print(f"\n{'=' * 60}")
    print("All maps generated successfully.")
    print("=" * 60)

GEOSPATIAL VISUALISATION PIPELINE

Creating Greenland Overview Map...
=== Bathymetry Data Loading ===
  CRS: EPSG:3996
  Greenland overview saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/greenland_overview_map.png

Creating Individual Glacier Detail Maps...
  Detail map saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/Helheim_Gletsjer_Comparison.png
  Detail map saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/Storebjoern_Comparison.png
  Detail map saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/Daugaard-Jensen_Gletsjer_Comparison.png
  Detail map saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/Waltershausen_Gletsjer_Comparison.png

All maps generated successfully.


## Section 3: Comparative Analysis of Greenland Glacier Summer (JJA) Dynamics: Ice Discharge, Runoff, and Submarine Melt Rate Visualization

This visualization system employs a multi-variable approach using glacier runoff and ice discharge data from Karlsson et al. (2023), with all hydrological variables converted from native units (m³/month) to m³/s for physical interpretability and consistency with the submarine melt rate parameterization. (Section 5) Individual glacier trajectories are represented as semi-transparent lines with region-coded coloring, using dark red at 40% opacity (alpha=0.4) for northeast (NE) glaciers and dark blue at 40% opacity for southeast (SE) glaciers. Regional aggregates are displayed as bold 3-point-width solid lines to enable clear sector-wide comparisons. The system implements intelligent annotations through auto-positioned labels featuring a coordinate-space buffer system for collision avoidance, with white-bordered text connected by directional arrows rendered at 50% opacity (alpha=0.5) using a '->' arrowstyle.

Technical implementation features include publication-grade 300DPI JPEG outputs with standardized filename conventions (regional_[Runoff|IceDischarge|SubmarineMeltRate]_comparison.png), powered by a dynamic layout engine. This engine provides configurable legend positioning (upper-left for runoff plots with adjustments for ice discharge displays), optimized reference grids using dotted lines at 30% opacity (alpha=0.3), and responsive title formatting with 14-point bold font and 15-padding. Robust data integrity safeguards are implemented through type validation during CSV ingestion, NaN-aware statistical aggregation, and precise region-specific data partitioning along the 69°N boundary separating NE and SE sectors.

The system generates dual visualizations for runoff and ice discharge through a shared rendering pipeline that maintains strict visual consistency. This is achieved via unified color schemes (NE=#8B0000 [darkred], SE=#00008B [darkblue]), synchronized temporal axes spanning 2010-2020 with integer-year ticks, and identical annotation styling using 10-point bold font within 0.3-padded rounded borders. The integrated analytical approach enables simultaneous macro/micro-scale examination through three innovative techniques: visual layering combining base individual glacier plots with regional mean overlays; contextual labeling with terminal-point annotations implementing value-space collision avoidance; and dynamic scaling with auto-adjusted y-axes in m³/s for each hydrological variable.

A third time-series visualization presents estimated submarine melt rates (m/day) computed using the Slater & Straneo (2022) buoyant plume parameterization (ṁ = 0.142 × Q⁰·³¹ × TF¹·¹⁹), where subglacial discharge Q (m³/s) is derived from Karlsson et al. (2023) runoff data and thermal forcing TF (°C) from ORAS5 300m-depth potential temperatures (Section 1). This figure follows the same dual-layer visualization approach—individual glacier trajectories with regional mean overlays—enabling direct visual comparison with ice discharge patterns. The conversion from m³/month to m³/s uses a standard month of 30.44 days (30.44 × 86,400 = 2,630,016 seconds/month).

In [12]:
# =============================================================================
# Section 3: Glacier Dynamics — Summer (JJA) Time Series Visualisation
# =============================================================================
# This script produces three time-series line plots comparing NE and SE
# Greenland glacier dynamics over 2010–2020 (summer JJA averages):
#
#   (1) Runoff (m³/s) — individual glaciers + regional horizontal means
#   (2) Ice Discharge (m³/s) — individual glaciers + regional means,
#       with a double-headed arrow annotating the inter-regional difference
#   (3) Submarine Melt Rate (m/day) — individual glaciers + regional means,
#       from Slater & Straneo (2022) parameterisation applied in Section 5
#
# Runoff and Ice Discharge source data are in m³/month (Karlsson et al., 2023).
# Conversion to m³/s: divide by (30.44 × 86400) = 2,630,016 seconds/month.
# Submarine Melt Rate is pre-computed in m/day by Section 5 Code 1.
#
# Dependencies:
#   - Karlsson et al. (2023) individual glacier CSVs (Runoff, Ice Discharge)
#   - individual_glaciers_summer.csv from Section 5 (Submarine Melt Rate)
#     → Section 5 must be executed first to generate this file
#
# Output: Three PNG figures saved to the plots/ directory
# =============================================================================

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.lines import Line2D

# =====================================================================
# Configuration
# =====================================================================

# Karlsson et al. (2023) individual glacier data files
data_paths = {
    "Helheim Gletsjer": "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/HelheimGletsjer_SE_D231.csv",
    "Storebjoern": "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/Storebjoern_SE_D299.csv",
    "Daugaard-Jensen Gletsjer": "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/Daugaard-JensenGletsjer_CE_D140.csv",
    "Waltershausen Gletsjer": "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/WaltershausenGletsjer_NE_D114.csv"
}

# Regional classification (divided at 69°N; Seale et al., 2011)
region_tags = {
    "Helheim Gletsjer": "SE",
    "Storebjoern": "SE",
    "Daugaard-Jensen Gletsjer": "NE",
    "Waltershausen Gletsjer": "NE"
}

output_folder = "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots"
os.makedirs(output_folder, exist_ok=True)

# Unit conversion constant: m³/month → m³/s
# Average month = 30.44 days; 30.44 × 86400 = 2,630,016 seconds
SECONDS_PER_MONTH = 30.44 * 86400

# Visual settings
REGION_COLORS = {'NE': 'darkred', 'SE': 'darkblue'}


# =====================================================================
# Data Loading and Preprocessing
# =====================================================================

def load_and_preprocess(path):
    """
    Load and clean a Karlsson et al. (2023) glacier CSV file.

    The raw CSVs have two header rows; this function extracts the
    correct column names, parses dates, and converts Runoff and
    IceDischarge columns to numeric types.

    Args:
        path (str): File path to the glacier CSV.

    Returns:
        pd.DataFrame: Cleaned dataframe with Date, Year, Month,
                      Runoff (m³/month), and IceDischarge (m³/month).
    """
    df = pd.read_csv(path)
    df_clean = df[2:].copy()
    df_clean.columns = df.iloc[1]
    df_clean.rename(columns={"Date_YYYY-MM": "Date"}, inplace=True)
    df_clean["Date"] = pd.to_datetime(df_clean["Date"])
    df_clean["Year"] = df_clean["Date"].dt.year
    df_clean["Month"] = df_clean["Date"].dt.month
    df_clean["Runoff"] = pd.to_numeric(df_clean["Runoff"], errors='coerce')
    df_clean["IceDischarge"] = pd.to_numeric(df_clean["IceDischarge"], errors='coerce')
    return df_clean


def get_summer_averages(df):
    """
    Compute summer (JJA) annual averages for Runoff and Ice Discharge.

    Filters for June, July, and August, then groups by year to
    calculate the mean of each variable in the original m³/month units.

    Args:
        df (pd.DataFrame): Preprocessed glacier dataframe.

    Returns:
        pd.DataFrame: Yearly JJA averages with columns Year, Runoff,
                      IceDischarge (all in m³/month).
    """
    summer_df = df[df["Month"].isin([6, 7, 8])]
    return summer_df.groupby("Year").agg({
        "Runoff": "mean",
        "IceDischarge": "mean"
    }).reset_index()


def get_regional_data(all_data):
    """
    Aggregate individual glacier data into NE and SE regional groups.

    For each region, combines all glacier time series and computes
    the inter-glacier mean per year. Used for plotting individual
    traces and regional mean horizontal lines.

    Args:
        all_data (dict): {glacier_name: summer_avg_dataframe}.

    Returns:
        dict: {region: {'individual': combined_df, 'mean': regional_mean_df}}.
    """
    regional_data = {}
    for region in ['NE', 'SE']:
        region_dfs = []
        for glacier, data in all_data.items():
            if region_tags[glacier] == region:
                df = data.copy()
                df['Glacier'] = glacier
                region_dfs.append(df)
        combined = pd.concat(region_dfs)
        regional_mean = combined.groupby('Year').mean(numeric_only=True).reset_index()
        regional_data[region] = {
            'individual': combined,
            'mean': regional_mean
        }
    return regional_data


# =====================================================================
# Figure 1 & 2: Runoff and Ice Discharge Line Plots
# =====================================================================

def create_combined_line_plot(all_data, regional_data, variable):
    """
    Create a time-series line plot for Runoff or Ice Discharge (m³/s).

    Plots individual glacier traces (thin, semi-transparent) and
    regional mean horizontal lines (thick, solid) for NE and SE.
    For Ice Discharge, a double-headed arrow annotates the
    inter-regional magnitude difference.

    Args:
        all_data (dict): Individual glacier summer average data.
        regional_data (dict): Regional aggregation from get_regional_data().
        variable (str): 'Runoff' or 'IceDischarge'.
    """
    plt.figure(figsize=(14, 8))

    # Format variable name for axis labels and title
    display_var = "Ice Discharge" if variable == "IceDischarge" else variable

    legend_elements = []

    # --- Individual glacier traces ---
    label_positions = {}
    for region in ['NE', 'SE']:
        color = REGION_COLORS[region]
        region_df = regional_data[region]['individual']

        for glacier in region_df['Glacier'].unique():
            glacier_data = region_df[region_df['Glacier'] == glacier]
            # Convert from m³/month to m³/s at plot time
            scaled_data = glacier_data[variable] / SECONDS_PER_MONTH
            plt.plot(glacier_data['Year'], scaled_data,
                     color=color, alpha=0.4, linewidth=1.5)

            # Annotated glacier name labels at end of each trace
            last_year = glacier_data['Year'].iloc[-1]
            last_value = scaled_data.iloc[-1]
            key = (last_year, round(last_value, 1))

            if key not in label_positions:
                y_offset = 0
                for existing_key in label_positions.keys():
                    if abs(existing_key[0] - last_year) < 1:
                        y_offset += 0.8

                plt.annotate(
                    glacier,
                    xy=(last_year, last_value),
                    xytext=(last_year + 0.5, last_value + y_offset),
                    color=color, fontsize=10, weight='bold',
                    arrowprops=dict(arrowstyle='->', color=color, alpha=0.5),
                    bbox=dict(boxstyle='round,pad=0.3',
                              facecolor='white', alpha=0.8, edgecolor=color)
                )
                label_positions[key] = True

        legend_elements.append(
            Line2D([0], [0], color=color, alpha=0.4, lw=1.5,
                   label=f'{region} Individual Glaciers'))

    # --- Regional mean horizontal lines ---
    regional_mean_values = {}
    for region in ['NE', 'SE']:
        color = REGION_COLORS[region]
        mean_data = regional_data[region]['mean']

        # Overall period mean, converted to m³/s
        regional_mean_value = mean_data[variable].mean() / SECONDS_PER_MONTH
        regional_mean_values[region] = regional_mean_value

        plt.axhline(y=regional_mean_value, color=color,
                     linewidth=3, linestyle='-', alpha=0.8)

        legend_elements.append(
            Line2D([0], [0], color=color, lw=3, linestyle='-',
                   label=f'{region} Regional Mean ({regional_mean_value:.2f} m\u00b3/s)'))

    # --- Difference annotation (Ice Discharge only) ---
    if variable == "IceDischarge":
        se_mean = regional_mean_values['SE']
        ne_mean = regional_mean_values['NE']
        difference = se_mean - ne_mean
        ratio = se_mean / ne_mean

        mid_year = 2015
        mid_value = (se_mean + ne_mean) / 2

        plt.annotate('',
                     xy=(mid_year, se_mean),
                     xytext=(mid_year, ne_mean),
                     arrowprops=dict(arrowstyle='<->',
                                     lw=2.5, color='black',
                                     shrinkA=0, shrinkB=0))

        plt.text(mid_year + 0.3, mid_value,
                 f'~{ratio:.1f}\u00d7 difference\n({difference:.1f} m\u00b3/s)',
                 fontsize=12, fontweight='bold', va='center',
                 bbox=dict(boxstyle='round,pad=0.5',
                           facecolor='yellow', alpha=0.7,
                           edgecolor='black', linewidth=2))

    # --- Styling ---
    plt.title(f"Regional Mean Summer (JJA) {display_var} Comparison (2010\u20132020)",
              fontsize=14, pad=15, weight='bold')
    plt.xlabel("Year", fontsize=12)
    plt.ylabel(f"{display_var} (m\u00b3/s)", fontsize=12)
    plt.xticks(range(2010, 2021))
    plt.grid(True, linestyle=':', alpha=0.3)

    legend_bbox = (1, 0.85) if variable == "IceDischarge" else (1, 1)
    plt.legend(handles=legend_elements, loc='upper left',
               bbox_to_anchor=legend_bbox, framealpha=1)

    plt.tight_layout(rect=[0, 0, 0.85, 1])
    save_path = os.path.join(output_folder, f"regional_{variable}_comparison.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"  Saved: {save_path}")


# =====================================================================
# Figure 3: Submarine Melt Rate Line Plot
# =====================================================================

def create_submarine_melt_plot():
    """
    Create a time-series line plot for Submarine Melt Rate (m/day).

    Reads pre-computed SubmarineDischarge values from the Section 5
    output CSV (individual_glaciers_summer.csv). Values are already
    in m/day, calculated using Slater & Straneo (2022):
        m_dot = 0.142 * Q^0.31 * TF^1.19
    where Q is subglacial discharge in m³/s.

    Visual style matches the Runoff and Ice Discharge line plots.
    Requires Section 5 Code 1 to have been executed first.
    """
    csv_path = '/Users/kyle/Documents/dissertation/Karlsson et al., 2023/tables/individual_glaciers_summer.csv'
    if not os.path.exists(csv_path):
        print(f"  CSV not found: {csv_path}")
        print("  Run Section 5 (data integration) first to generate this file.")
        return

    combined_df = pd.read_csv(csv_path)

    if 'SubmarineDischarge' not in combined_df.columns:
        print("  'SubmarineDischarge' column not found in CSV.")
        print("  Ensure Section 5 code includes submarine melt rate calculation.")
        return

    plt.figure(figsize=(14, 8))
    legend_elements = []
    label_positions = {}

    # Prepare regional data structures
    regional_data = {}
    for region in ['NE', 'SE']:
        region_df = combined_df[combined_df['Region'] == region].copy()
        regional_mean = region_df.groupby('Year')['SubmarineDischarge'].mean().reset_index()
        regional_data[region] = {
            'individual': region_df,
            'mean': regional_mean
        }

    # --- Individual glacier traces ---
    for region in ['NE', 'SE']:
        color = REGION_COLORS[region]
        region_df = regional_data[region]['individual']

        for glacier in region_df['Glacier'].unique():
            glacier_data = region_df[region_df['Glacier'] == glacier]
            plt.plot(glacier_data['Year'], glacier_data['SubmarineDischarge'],
                     color=color, alpha=0.4, linewidth=1.5)

            # Annotated glacier name labels
            last_year = glacier_data['Year'].iloc[-1]
            last_value = glacier_data['SubmarineDischarge'].iloc[-1]
            key = (last_year, round(last_value, 1))

            if key not in label_positions:
                y_offset = 0
                for existing_key in label_positions.keys():
                    if abs(existing_key[0] - last_year) < 1:
                        y_offset += 0.8

                plt.annotate(
                    glacier,
                    xy=(last_year, last_value),
                    xytext=(last_year + 0.5, last_value + y_offset),
                    color=color, fontsize=10, weight='bold',
                    arrowprops=dict(arrowstyle='->', color=color, alpha=0.5),
                    bbox=dict(boxstyle='round,pad=0.3',
                              facecolor='white', alpha=0.8, edgecolor=color)
                )
                label_positions[key] = True

        legend_elements.append(
            Line2D([0], [0], color=color, alpha=0.4, lw=1.5,
                   label=f'{region} Individual Glaciers'))

    # --- Regional mean horizontal lines ---
    regional_mean_values = {}
    for region in ['NE', 'SE']:
        color = REGION_COLORS[region]
        mean_val = regional_data[region]['mean']['SubmarineDischarge'].mean()
        regional_mean_values[region] = mean_val

        plt.axhline(y=mean_val, color=color,
                     linewidth=3, linestyle='-', alpha=0.8)
        legend_elements.append(
            Line2D([0], [0], color=color, lw=3, linestyle='-',
                   label=f'{region} Regional Mean ({mean_val:.2f} m/day)'))

    # --- Difference annotation ---
    se_mean = regional_mean_values['SE']
    ne_mean = regional_mean_values['NE']
    difference = se_mean - ne_mean
    ratio = se_mean / ne_mean if ne_mean > 0 else float('inf')

    mid_year = 2015
    mid_value = (se_mean + ne_mean) / 2

    plt.annotate('',
                 xy=(mid_year, se_mean),
                 xytext=(mid_year, ne_mean),
                 arrowprops=dict(arrowstyle='<->',
                                 lw=2.5, color='black',
                                 shrinkA=0, shrinkB=0))

    plt.text(mid_year + 0.3, mid_value,
             f'~{ratio:.1f}\u00d7 difference\n({difference:.2f} m/day)',
             fontsize=12, fontweight='bold', va='center',
             bbox=dict(boxstyle='round,pad=0.5',
                       facecolor='yellow', alpha=0.7,
                       edgecolor='black', linewidth=2))

    # --- Styling ---
    plt.title("Regional Mean Summer (JJA) Submarine Melt Rate Comparison (2010\u20132020)",
              fontsize=14, pad=15, weight='bold')
    plt.xlabel("Year", fontsize=12)
    plt.ylabel("Submarine Melt Rate (m/day)", fontsize=12)
    plt.xticks(range(2010, 2021))
    plt.grid(True, linestyle=':', alpha=0.3)

    plt.legend(handles=legend_elements, loc='upper left',
               bbox_to_anchor=(1, 1), framealpha=1)

    plt.tight_layout(rect=[0, 0, 0.85, 1])
    save_path = os.path.join(output_folder, "regional_SubmarineMeltRate_comparison.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"  Saved: {save_path}")
    print(f"\n  Submarine Melt Rate Statistics:")
    print(f"    SE Regional Mean: {se_mean:.2f} m/day")
    print(f"    NE Regional Mean: {ne_mean:.2f} m/day")
    print(f"    Difference: {difference:.2f} m/day (~{ratio:.1f}x)")


# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    print("=" * 60)
    print("GLACIER DYNAMICS TIME SERIES PIPELINE")
    print("=" * 60)

    # Step 1: Load and preprocess individual glacier data
    print("\nLoading Karlsson et al. (2023) glacier data...")
    all_glacier_data = {}
    for glacier, path in data_paths.items():
        print(f"  Processing {glacier}...")
        df = load_and_preprocess(path)
        summer_avg = get_summer_averages(df)
        all_glacier_data[glacier] = summer_avg

    # Step 2: Aggregate into regional groups
    regional_data = get_regional_data(all_glacier_data)

    # Step 3: Generate Runoff and Ice Discharge line plots (m³/s)
    print("\nGenerating line plots...")
    create_combined_line_plot(all_glacier_data, regional_data, "Runoff")
    create_combined_line_plot(all_glacier_data, regional_data, "IceDischarge")

    # Step 4: Generate Submarine Melt Rate line plot (m/day)
    # Requires individual_glaciers_summer.csv from Section 5
    print("\nGenerating Submarine Melt Rate plot...")
    create_submarine_melt_plot()

    print(f"\n{'=' * 60}")
    print("All time-series plots generated successfully.")
    print("=" * 60)

GLACIER DYNAMICS TIME SERIES PIPELINE

Loading Karlsson et al. (2023) glacier data...
  Processing Helheim Gletsjer...
  Processing Storebjoern...
  Processing Daugaard-Jensen Gletsjer...
  Processing Waltershausen Gletsjer...

Generating line plots...
  Saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/regional_Runoff_comparison.png
  Saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/regional_IceDischarge_comparison.png

Generating Submarine Melt Rate plot...
  Saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/regional_SubmarineMeltRate_comparison.png

  Submarine Melt Rate Statistics:
    SE Regional Mean: 7.19 m/day
    NE Regional Mean: 1.73 m/day
    Difference: 5.46 m/day (~4.2x)

All time-series plots generated successfully.


## Section 4: Ocean Potential Temperature Analysis at Greenland Glacier Margins (2010-2020)

This study implements a comprehensive analytical workflow to examine summer (JJA) potential temperature patterns near four major Greenland glaciers using ORAS5 reanalysis data (CDS, 2021). The methodology begins with systematic data acquisition, loading monthly 3D temperature fields from NetCDF files while incorporating manually validated data points for July 2015 derived from the nearest available grid cells in the original ORAS5 file (votemper_control_monthly_highres_3D_201507_OPER_v0.1.nc), as identified through the spatial search algorithm detailed in Section 1. Each processing step includes rigorous coordinate system validation, verifying that data falls within expected Greenlandic boundaries (50-85°N, -75--10°E) and automatically correcting for latitude axis orientation when necessary. This visualization presents the 300m-depth potential temperature data extracted in Section 1 as regional time series, enabling visual assessment of interannual variability and NE/SE thermal contrasts.

The core analysis extracts potential temperatures at glacier-adjacent ocean grid points through an optimized grid-point selection algorithm that minimizes Euclidean distance between dataset coordinates and predefined glacier termini positions, building directly on the methodology established in Section 1. Spatial validation metrics are computed with geodesic distance calculations (reported to 2 decimal places in kilometers) from target locations for each data point. Summer (JJA) means are calculated by averaging June-August temperatures, with the resulting time series undergoing quality checks for completeness and physical plausibility.

Visualization employs a dual-layer approach combining individual glacier trends with regional aggregates. Semi-transparent lines (α=0.4) in region-specific colors (NE=#8B0000 [darkred], SE=#00008B [darkblue]) represent single-glacier observations, while bold solid lines show sector-wide averages. An intelligent labeling system positions glacier identifiers at line endpoints with automatic collision avoidance, using white-background annotations connected by directional arrows. The plotting framework includes zero-reference lines, decade-year ticks, and dynamic legend placement optimized for readability.

Technical implementation features robust error handling with formatted runtime messages (including filename/year tags for processing alerts), validation warnings, and configurable output settings. High-resolution (300 DPI) PNG visualizations are systematically saved with standardized nomenclature ("regional_temperature_comparison.png"), maintaining consistency with other sections' output formats. This integrated approach enables simultaneous examination of local thermal anomalies and regional climate patterns, with Section 1's extraction methodology and the current time-series visualization together providing insights into interannual variability of subsurface oceanographic conditions.

In [13]:
# =============================================================================
# Section 4: Ocean Temperature — Summer (JJA) Time Series Visualisation
# =============================================================================
# This script extracts 300m-depth ocean potential temperature from ORAS5
# reanalysis (CDS, 2021) and produces a regional comparison line plot
# for NE and SE Greenland glaciers over 2010–2020.
#
# Unlike Section 1 Code 2 (which outputs a CSV), this script combines
# data extraction and visualisation in a single pipeline. It reads
# individual ORAS5 NetCDF files for each JJA month, extracts temperature
# at the replacement ocean grid point for each glacier, and plots:
#   - Individual glacier traces (thin, semi-transparent)
#   - Regional mean horizontal lines (thick, solid)
#   - Inter-regional temperature difference annotation
#
# Grid point matching uses Euclidean distance minimisation on the ORAS5
# curvilinear grid (nav_lat, nav_lon), consistent with Section 1 Code 2.
#
# Note: July 2015 uses manually verified values from Section 1 extraction
# due to a file-format inconsistency in the OPER/CONS transition period.
#
# Data source: ORAS5 ocean reanalysis (CDS, 2021)
# Target depth: 300m (dynamically resolved via depth index search)
# =============================================================================

import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# =====================================================================
# Configuration
# =====================================================================

base_dir = "/Users/kyle/Documents/dissertation/ORAS5/votemper"
output_folder = "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots"
os.makedirs(output_folder, exist_ok=True)

# Replacement coordinates: nearest valid ORAS5 ocean grid points at 300m
# depth, identified by Section 1 Code 1
replacement_locations = {
    "Helheim Gletsjer":         {"lat": 65.2358, "lon": -37.9326},   
    "Storebjoern":              {"lat": 62.9264, "lon": -40.8969},   
    "Daugaard-Jensen Gletsjer": {"lat": 71.1908, "lon": -25.1324},   
    "Waltershausen Gletsjer":   {"lat": 72.0131, "lon": -22.0269}    
}

# Regional classification (divided at 69°N; Seale et al., 2011)
region_tags = {
    "Helheim Gletsjer": "SE",
    "Storebjoern": "SE",
    "Daugaard-Jensen Gletsjer": "NE",
    "Waltershausen Gletsjer": "NE"
}

# Visual settings
REGION_COLORS = {'NE': 'darkred', 'SE': 'darkblue'}

# Manually verified July 2015 temperatures from Section 1 extraction.
# Used as fallback for the OPER/CONS file-format transition period.
MANUAL_DATA_JULY_2015 = {
    "Helheim Gletsjer": 4.33,
    "Storebjoern": 4.66,
    "Daugaard-Jensen Gletsjer": 0.01,
    "Waltershausen Gletsjer": -0.04
}


# =====================================================================
# Main Pipeline: Data Extraction + Visualisation
# =====================================================================

def create_combined_temperature_plot():
    """
    Extract 300m-depth ORAS5 temperature for each glacier across
    2010–2020 JJA months and produce a regional comparison line plot.

    Extraction method:
        1. For each year and JJA month, open the corresponding NetCDF file.
        2. Find the depth index closest to 300m (dynamic search).
        3. Locate the nearest grid point to each glacier's replacement
           coordinates via Euclidean distance minimisation.
        4. Extract the temperature value; skip NaN (land-masked) cells.

    The resulting plot shows individual glacier traces and regional
    mean horizontal lines with an inter-regional difference annotation.
    """
    # --- Data extraction ---
    all_glacier_data = {}

    for glacier, loc in replacement_locations.items():
        print(f"\nProcessing {glacier} ({loc['lat']:.4f}°N, {loc['lon']:.4f}°E)...")
        yearly_data = []

        for year in range(2010, 2021):
            summer_temps = []

            for month in [6, 7, 8]:
                # Use manually verified data for July 2015
                if year == 2015 and month == 7:
                    temp = MANUAL_DATA_JULY_2015.get(glacier, np.nan)
                    if not np.isnan(temp):
                        summer_temps.append(temp)
                    continue

                # Determine file prefix (CONS before June 2015, OPER after)
                prefix = "CONS" if year < 2015 or (year == 2015 and month < 6) else "OPER"
                file_name = f"votemper_control_monthly_highres_3D_{year}{month:02d}_{prefix}_v0.1.nc"
                file_path = os.path.join(base_dir, file_name)

                if not os.path.exists(file_path):
                    continue

                try:
                    with xr.open_dataset(file_path) as ds:
                        # Find depth index closest to 300m (dynamic, not hardcoded)
                        depth_idx = np.abs(ds.deptht.values - 300).argmin()

                        # Find nearest grid point via Euclidean distance
                        dist = (ds.nav_lat - loc['lat'])**2 + (ds.nav_lon - loc['lon'])**2
                        y_idx, x_idx = np.unravel_index(
                            np.argmin(dist.values), dist.shape
                        )

                        # Extract temperature at 300m depth
                        temp = ds.votemper.isel(
                            time_counter=0, deptht=depth_idx,
                            y=y_idx, x=x_idx
                        ).values.item()

                        if not np.isnan(temp):
                            summer_temps.append(temp)

                except Exception as e:
                    print(f"  Error ({year}-{month:02d}): {e}")

            if summer_temps:
                yearly_data.append({
                    "Year": year,
                    "Temp_Avg": np.mean(summer_temps),
                    "Glacier": glacier,
                    "Region": region_tags[glacier]
                })

        if yearly_data:
            all_glacier_data[glacier] = pd.DataFrame(yearly_data)
            print(f"  {len(yearly_data)} years extracted "
                  f"(mean: {np.mean([d['Temp_Avg'] for d in yearly_data]):.2f}°C)")

    # --- Regional aggregation ---
    regional_data = {}
    for region in ['NE', 'SE']:
        region_dfs = []
        for glacier, data in all_glacier_data.items():
            if region_tags[glacier] == region:
                region_dfs.append(data.copy())
        combined = pd.concat(region_dfs)
        regional_mean = combined.groupby('Year').agg(
            {'Temp_Avg': 'mean'}
        ).reset_index()
        regional_data[region] = {
            'individual': combined,
            'mean': regional_mean
        }

    # --- Plotting ---
    plt.figure(figsize=(14, 8))
    legend_elements = []

    # Per-glacier label positioning to avoid overlap
    label_properties = {
        "Waltershausen Gletsjer":   {"offset": -0.3, "ha": "left"},
        "Daugaard-Jensen Gletsjer": {"offset":  0.8, "ha": "left"},
        "Helheim Gletsjer":         {"offset": -0.3, "ha": "left"},
        "Storebjoern":              {"offset":  0.7, "ha": "left"},
    }

    # Individual glacier traces
    for region in ['NE', 'SE']:
        color = REGION_COLORS[region]
        region_df = regional_data[region]['individual']

        for glacier in region_df['Glacier'].unique():
            glacier_data = region_df[region_df['Glacier'] == glacier]
            plt.plot(glacier_data['Year'], glacier_data['Temp_Avg'],
                     color=color, alpha=0.4, linewidth=1.5)

            # Annotated glacier name labels at end of trace
            last_year = glacier_data['Year'].iloc[-1]
            last_value = glacier_data['Temp_Avg'].iloc[-1]

            props = label_properties.get(glacier, {"offset": 0, "ha": "center"})
            y_offset = props["offset"]
            ha = props["ha"]

            plt.annotate(
                glacier,
                xy=(last_year, last_value),
                xytext=(last_year + (0.3 if ha == "left" else -0.3),
                        last_value + y_offset),
                color=color, fontsize=10, weight='bold',
                ha=ha, va='center',
                arrowprops=dict(arrowstyle='->', color=color, alpha=0.5,
                                relpos=(0.5 if ha == "center"
                                        else (0.2 if ha == "left" else 0.8))),
                bbox=dict(boxstyle='round,pad=0.3',
                          facecolor='white', alpha=0.8, edgecolor=color)
            )

        legend_elements.append(
            Line2D([0], [0], color=color, alpha=0.4, lw=1.5,
                   label=f'{region} Individual Glaciers'))

    # Regional mean horizontal lines
    regional_mean_values = {}
    for region in ['NE', 'SE']:
        color = REGION_COLORS[region]
        mean_data = regional_data[region]['mean']

        regional_mean_value = mean_data['Temp_Avg'].mean()
        regional_mean_values[region] = regional_mean_value

        plt.axhline(y=regional_mean_value, color=color,
                     linewidth=3, linestyle='-', alpha=0.8)

        legend_elements.append(
            Line2D([0], [0], color=color, lw=3, linestyle='-',
                   label=f'{region} Regional Mean ({regional_mean_value:.2f}\u00b0C)'))

    # Inter-regional temperature difference annotation
    se_mean = regional_mean_values['SE']
    ne_mean = regional_mean_values['NE']
    difference = se_mean - ne_mean

    mid_year = 2015
    mid_value = (se_mean + ne_mean) / 2

    plt.annotate('',
                 xy=(mid_year, se_mean),
                 xytext=(mid_year, ne_mean),
                 arrowprops=dict(arrowstyle='<->',
                                 lw=2.5, color='black',
                                 shrinkA=0, shrinkB=0))

    plt.text(mid_year + 0.3, mid_value,
             f'{difference:.2f}\u00b0C difference',
             fontsize=12, fontweight='bold', va='center',
             bbox=dict(boxstyle='round,pad=0.5',
                       facecolor='yellow', alpha=0.7,
                       edgecolor='black', linewidth=2))

    # Styling
    plt.title("Regional Mean Summer (JJA) Potential Temperature at 300m Depth (2010\u20132020)",
              fontsize=14, pad=15, weight='bold')
    plt.xlabel("Year", fontsize=12)
    plt.ylabel("Temperature (\u00b0C)", fontsize=12)
    plt.xticks(range(2010, 2021))
    plt.grid(True, linestyle=':', alpha=0.3)
    plt.axhline(0, color='black', linewidth=0.8, zorder=0)

    plt.legend(handles=legend_elements, loc='upper left',
               bbox_to_anchor=(1, 0.5), framealpha=1)

    plt.tight_layout(rect=[0, 0, 0.85, 1])
    save_path = os.path.join(output_folder, "regional_temperature_comparison.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"\n  Saved: {save_path}")
    print(f"\n  Temperature Statistics:")
    print(f"    SE Regional Mean: {se_mean:.2f}\u00b0C")
    print(f"    NE Regional Mean: {ne_mean:.2f}\u00b0C")
    print(f"    Difference: {difference:.2f}\u00b0C")


# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    print("=" * 60)
    print("OCEAN TEMPERATURE TIME SERIES PIPELINE")
    print("=" * 60)
    create_combined_temperature_plot()
    print(f"\n{'=' * 60}")
    print("Temperature analysis completed.")
    print("=" * 60)

OCEAN TEMPERATURE TIME SERIES PIPELINE

Processing Helheim Gletsjer (65.2358°N, -37.9326°E)...
  11 years extracted (mean: 4.66°C)

Processing Storebjoern (62.9264°N, -40.8969°E)...
  11 years extracted (mean: 4.85°C)

Processing Daugaard-Jensen Gletsjer (71.1908°N, -25.1324°E)...
  11 years extracted (mean: -0.01°C)

Processing Waltershausen Gletsjer (72.0131°N, -22.0269°E)...
  11 years extracted (mean: 0.12°C)

  Saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/regional_temperature_comparison.png

  Temperature Statistics:
    SE Regional Mean: 4.75°C
    NE Regional Mean: 0.06°C
    Difference: 4.70°C

Temperature analysis completed.


## Section 5: Integrated Glacier-Ocean Data Processing Pipeline

These scripts combine three key components: glacier discharge processing, ocean temperature analysis, and statistical correlation evaluation. The analysis focuses on four major Greenlandic glaciers (Helheim, Storebjoern, Daugaard-Jensen, and Waltershausen) using glacier runoff and ice discharge from Karlsson et al. (2023) and ORAS5 reanalysis data (CDS, 2021). For discharge measurements, the script processes runoff and ice discharge data by calculating summer (June-August) averages from 2010 to 2020, with careful handling of unit conversions and missing values.

Central to this pipeline is the computation of submarine melt rates using the buoyant plume parameterization from Slater & Straneo (2022): ṁ = 0.142 × Q⁰·³¹ × TF¹·¹⁹, where Q represents subglacial discharge (m³/s) and TF represents thermal forcing (°C), defined as ocean potential temperature minus the pressure-dependent freezing point (−1.9°C at ~300m depth). Critically, the raw runoff data from Karlsson et al. (2023) is provided in m³/month and must be converted to m³/s by dividing by 2,630,016 (= 30.44 days × 86,400 s/day) before application in the parameterization. The resulting melt rate estimates are in m/day, consistent with the parameterization's calibration against LeConte Glacier observations (Jackson et al., 2020).

The temperature analysis utilizes ORAS5 ocean reanalysis data (CDS, 2021), which employs different data production systems across the study period. As specified in the ORAS5 documentation, the dataset transitions from Consolidated (CONS) to Operational (OPER) systems, requiring distinct file naming conventions. The script automatically handles this transition by using CONS prefix files for 2010-2014 and January-May 2015, then switching to OPER prefix for June 2015 onward. Potential temperature values are extracted from the nearest ocean grid points to each glacier's terminus location, with spatial matching through Euclidean distance minimization on the ORAS5 curvilinear grid, consistent with Section 1. The temperature extraction process includes depth-specific selection at 300m and spatial matching through minimum distance calculations between glacier coordinates and model grid points.

Statistical analysis examines relationships between estimated submarine melt rates (m/day) and observed ice discharge (m³/s) through Pearson correlation tests at both individual glacier and regional (NE vs SE) scales. Additional correlations between ocean temperature and runoff/ice discharge are computed for completeness. The correlations are evaluated at both individual glacier and regional levels (Northeast vs. Southeast Greenland), with results reported to three decimal places for both coefficients and p-values. Visualization outputs include formatted tables and three separate regression plots: (1) runoff vs. temperature, (2) ice discharge vs. temperature, and (3) submarine melt rate vs. ice discharge—the last being the study's primary analytical figure featuring scatter points with dashed trendlines (NE: #d62728, SE: #1f77b4) against 70% opacity dotted reference grids. The system generates publication-ready 300DPI PNG outputs with standardized dimensions, maintaining methodological transparency through preserved intermediate processing steps and clear connections between raw data sources, analytical methods, and final outputs, while adhering to the project's formatting requirements.

In [14]:
# =============================================================================
# Section 5, Code 1: Integrated Glacier-Ocean Data Processing Pipeline
# =============================================================================
# This script combines three key components:
# (1) Glacier discharge processing (Runoff & Ice Discharge)
# (2) Ocean temperature analysis (ORAS5 300m depth)
# (3) Submarine melt rate calculation using Slater & Straneo (2022)
#
# Parameterization: ṁ = 0.142 × Q^0.31 × TF^1.19
#   where Q = subglacial discharge (m³/s), TF = thermal forcing (°C)
#   Output: submarine melt rate in m/day
#
# CRITICAL UNIT CONVERSION:
#   Raw Runoff from Karlsson et al. (2023) is in m³/month.
#   Must convert to m³/s before applying the parameterization:
#   Q (m³/s) = Runoff (m³/month) / (30.44 × 86400)
# =============================================================================

import os
import numpy as np
import pandas as pd
import xarray as xr
from geopy.distance import geodesic

# Configuration
output_folder = "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/tables"
os.makedirs(output_folder, exist_ok=True)

# 1. Runoff and Ice Discharge data paths (Source: Karlsson et al., 2023)
discharge_paths = {
    "Helheim Gletsjer": "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/HelheimGletsjer_SE_D231.csv",
    "Storebjoern": "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/Storebjoern_SE_D299.csv",
    "Daugaard-Jensen Gletsjer": "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/Daugaard-JensenGletsjer_CE_D140.csv",
    "Waltershausen Gletsjer": "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/WaltershausenGletsjer_NE_D114.csv"
}

# 2. Potential Temperature extraction coordinates (Source: CDS, 2021; ORAS5)
replacement_locations = {
    "Helheim Gletsjer":         {"lat": 65.2358, "lon": -37.9326},   
    "Storebjoern":              {"lat": 62.9264, "lon": -40.8969},   
    "Daugaard-Jensen Gletsjer": {"lat": 71.1908, "lon": -25.1324},   
    "Waltershausen Gletsjer":   {"lat": 72.0131, "lon": -22.0269}    
}

# Regional classification divided at 69°N (Seale et al., 2011)
region_tags = {
    "Helheim Gletsjer": "SE",
    "Storebjoern": "SE",
    "Daugaard-Jensen Gletsjer": "NE",
    "Waltershausen Gletsjer": "NE"
}

# Unit conversion constant: m³/month → m³/s
# Average month = 30.44 days × 86400 seconds/day = 2,630,016 seconds
SECONDS_PER_MONTH = 30.44 * 86400


# =============================================================================
# 3. Runoff / Ice Discharge Processing (Source: Karlsson et al., 2023)
# =============================================================================

def process_discharge_data():
    """
    Load and process monthly runoff and ice discharge data for four glaciers.

    Computes summer (JJA) averages for each year from 2010 to 2020.
    Raw values remain in m³/month; unit conversion is applied downstream.

    Returns:
        pd.DataFrame: Summer averages with columns Glacier, Region, Year,
                      Runoff (m³/month), IceDischarge (m³/month).
    """
    all_discharge_data = []
    
    for glacier, path in discharge_paths.items():
        df = pd.read_csv(path)
        df_clean = df[2:].copy()
        df_clean.columns = df.iloc[1]
        df_clean.rename(columns={"Date_YYYY-MM": "Date"}, inplace=True)
        df_clean["Date"] = pd.to_datetime(df_clean["Date"])
        df_clean["Year"] = df_clean["Date"].dt.year
        df_clean["Month"] = df_clean["Date"].dt.month
        df_clean["Runoff"] = pd.to_numeric(df_clean["Runoff"], errors='coerce')
        df_clean["IceDischarge"] = pd.to_numeric(df_clean["IceDischarge"], errors='coerce')
        
        # Calculate summer (JJA) average
        summer_avg = df_clean[df_clean["Month"].isin([6, 7, 8])].groupby("Year").agg({
            "Runoff": "mean",
            "IceDischarge": "mean"
        }).reset_index().round(2)
        
        summer_avg["Glacier"] = glacier
        summer_avg["Region"] = region_tags[glacier]
        all_discharge_data.append(summer_avg)
    
    return pd.concat(all_discharge_data)


# =============================================================================
# 4. Potential Temperature Processing (Source: CDS, 2021; ORAS5 at 300m depth)
# =============================================================================

def process_temperature_data():
    """
    Extract summer (JJA) ocean potential temperature at 300m depth from ORAS5
    reanalysis for each glacier's ocean data point (2010–2020).

    The ORAS5 dataset transitions from Consolidated (CONS) to Operational (OPER)
    production system: CONS for 2010–2014 and Jan–May 2015, OPER from Jun 2015.

    July 2015 uses manually validated values derived from the Section 1
    spatial search algorithm applied to the original ORAS5 file.

    Grid point matching uses Euclidean distance minimisation on the ORAS5
    curvilinear grid (nav_lat, nav_lon), consistent with Section 1 Code 2
    and Section 4.

    Returns:
        pd.DataFrame: Yearly JJA temperature data with columns Glacier,
                      Region, Year, Temp_Avg, Temp_Min, Temp_Max,
                      Temp_Data_Points.
    """
    base_dir = "/Users/kyle/Documents/dissertation/ORAS5/votemper"
    
    # Manual validation data for July 2015 (Source: Section 1 ORAS5 extraction)
    manual_data = {
        2015: {
            "Helheim Gletsjer": 4.33,
            "Storebjoern": 4.66,
            "Daugaard-Jensen Gletsjer": 0.01,
            "Waltershausen Gletsjer": -0.04
        }
    }
    
    all_temp_data = []

    for glacier, loc in replacement_locations.items():
        yearly_data = []
        
        for year in range(2010, 2021):
            summer_temps = []
            
            for month in [6, 7, 8]:
                if year == 2015 and month == 7:
                    # Use manually validated value for July 2015
                    temp = manual_data[2015].get(glacier, np.nan)
                else:
                    # Determine file prefix based on ORAS5 production system
                    prefix = "CONS" if year < 2015 or (year == 2015 and month < 6) else "OPER"
                    file_name = f"votemper_control_monthly_highres_3D_{year}{month:02d}_{prefix}_v0.1.nc"
                    file_path = os.path.join(base_dir, file_name)
                    
                    if os.path.exists(file_path):
                        try:
                            with xr.open_dataset(file_path) as ds:
                                # Find nearest grid point via Euclidean distance
                                # (consistent with Section 1 Code 2 and Section 4)
                                dist = (ds.nav_lat - loc['lat'])**2 + (ds.nav_lon - loc['lon'])**2
                                y_idx, x_idx = np.unravel_index(np.argmin(dist.values), dist.shape)
                                
                                # Extract temperature at 300m depth (dynamic index)
                                depth_idx = np.abs(ds.deptht.values - 300).argmin()
                                temp = ds.votemper.isel(
                                    time_counter=0, deptht=depth_idx,
                                    y=y_idx, x=x_idx
                                ).values.item()
                                
                        except Exception as e:
                            print(f"  Error processing {file_name}: {e}")
                            temp = np.nan
                    else:
                        temp = np.nan
                        print(f"  File not found: {file_name}")
                
                if not np.isnan(temp):
                    summer_temps.append(temp)
            
            if summer_temps:
                yearly_data.append({
                    "Year": year,
                    "Temp_Avg": np.mean(summer_temps).round(3),
                    "Temp_Min": np.min(summer_temps).round(3),
                    "Temp_Max": np.max(summer_temps).round(3),
                    "Temp_Data_Points": len(summer_temps),
                })
        
        if yearly_data:
            temp_df = pd.DataFrame(yearly_data)
            temp_df["Glacier"] = glacier
            temp_df["Region"] = region_tags[glacier]
            all_temp_data.append(temp_df)
            print(f"  Processed {glacier}: {len(yearly_data)} years of data")
    
    final_df = pd.concat(all_temp_data, ignore_index=True) if all_temp_data else pd.DataFrame()
    print(f"\n  Total data collected: {len(final_df)} yearly averages")
    return final_df


# =============================================================================
# 5. Submarine Melt Rate Calculation (Slater & Straneo, 2022)
# =============================================================================

def calculate_submarine_discharge(df):
    """
    Calculate submarine melt rate (m/day) using buoyant plume parameterization:
        ṁ = 0.142 × Q^0.31 × TF^1.19

    where:
        Q  = subglacial discharge in m³/s (converted from m³/month)
        TF = thermal forcing in °C (ocean temperature minus freezing point)

    The freezing point at ~300m depth is approximately -1.9°C.

    Validation reference (Donald Slater):
        Q = 600 m³/s, TF = 6.5°C → ṁ ≈ 9.5 m/day
        Calculated: 0.142 × 600^0.31 × 6.5^1.19 = 9.57 m/day

    Args:
        df (pd.DataFrame): Merged discharge and temperature data.

    Returns:
        pd.DataFrame: Input dataframe with added TF and SubmarineDischarge columns.
    """
    df_calc = df.copy()
    
    # Thermal Forcing (TF) = Ocean Temperature - Pressure-Dependent Freezing Point
    freezing_point = -1.9  # °C at ~300m depth
    df_calc['TF'] = df_calc['Temp_Avg'] - freezing_point
    
    # Convert Runoff from m³/month to m³/s
    df_calc['Runoff_m3s'] = df_calc['Runoff'] / SECONDS_PER_MONTH
    
    # Submarine Melt Rate (m/day)
    df_calc['SubmarineDischarge'] = np.where(
        (df_calc['Runoff_m3s'] > 0) & (df_calc['TF'] > 0),
        0.142 * (df_calc['Runoff_m3s'] ** 0.31) * (df_calc['TF'] ** 1.19),
        0
    )
    
    df_calc['TF'] = df_calc['TF'].round(3)
    df_calc['SubmarineDischarge'] = df_calc['SubmarineDischarge'].round(2)
    
    # Drop intermediate conversion column (not needed in final output)
    df_calc = df_calc.drop(columns=['Runoff_m3s'])
    
    # Print summary statistics
    print("\n=== Submarine Melt Rate Summary (m/day) ===")
    for region in ['NE', 'SE']:
        region_data = df_calc[df_calc['Region'] == region]['SubmarineDischarge']
        print(f"  {region}: Mean={region_data.mean():.2f}, "
              f"Min={region_data.min():.2f}, Max={region_data.max():.2f}")
    
    return df_calc


# =============================================================================
# 6. Main Processing Pipeline
# =============================================================================

def main():
    """
    Execute the full glacier-ocean data integration pipeline.

    Steps:
        1. Process discharge data (Karlsson et al., 2023)
        2. Process temperature data (ORAS5 at 300m depth)
        3. Merge discharge and temperature datasets
        4. Calculate submarine melt rates (Slater & Straneo, 2022)
        5. Save individual glacier and regional mean CSVs
    """
    # Step 1: Process discharge data (Runoff & Ice Discharge)
    print("=== Processing Discharge Data ===")
    discharge_df = process_discharge_data()
    
    # Step 2: Process temperature data (ORAS5 300m depth)
    print("\n=== Processing Temperature Data ===")
    temp_df = process_temperature_data()
    
    # Step 3: Merge discharge and temperature data
    combined_df = pd.merge(discharge_df, temp_df, on=["Glacier", "Region", "Year"], how="outer")
    
    # Step 4: Calculate submarine melt rates (m/day)
    # CRITICAL: Runoff is converted from m³/month to m³/s inside this function
    print("\n=== Calculating Submarine Melt Rates ===")
    combined_df = calculate_submarine_discharge(combined_df)
    
    # Step 5: Reorder columns for output CSV
    column_order = [
        "Glacier", "Region", "Year", 
        "Runoff", "IceDischarge",                   # m³/month (raw from Karlsson et al., 2023)
        "Temp_Avg", "Temp_Min", "Temp_Max",         # °C at 300m depth (ORAS5)
        "TF",                                        # °C (thermal forcing = Temp - freezing point)
        "SubmarineDischarge",                        # m/day (Slater & Straneo, 2022)
        "Temp_Data_Points"                           # Number of JJA months with valid data
    ]
    combined_df = combined_df[column_order]
    
    # Step 6: Save individual glacier data
    individual_path = os.path.join(output_folder, "individual_glaciers_summer.csv")
    combined_df.to_csv(individual_path, index=False)
    print(f"\n  Individual glacier data saved: {individual_path}")
    
    # Step 7: Calculate and save regional means
    regional_means = combined_df.groupby(['Region', 'Year']).agg({
        'Runoff': 'mean',
        'IceDischarge': 'mean',
        'Temp_Avg': 'mean',
        'Temp_Min': 'min',
        'Temp_Max': 'max',
        'TF': 'mean',
        'SubmarineDischarge': 'mean',
        'Temp_Data_Points': 'sum'
    }).reset_index().round(2)
    
    regional_path = os.path.join(output_folder, "regional_means_summer.csv")
    regional_means.to_csv(regional_path, index=False)
    print(f"  Regional means data saved: {regional_path}")
    
    # Print final data overview
    print(f"\n{'='*60}")
    print("DATA SUMMARY")
    print(f"{'='*60}")
    print(f"  Total records: {len(combined_df)}")
    print(f"  Glaciers: {combined_df['Glacier'].nunique()}")
    print(f"  Years: {combined_df['Year'].min()}-{combined_df['Year'].max()}")
    print(f"  Columns: {list(combined_df.columns)}")
    print(f"\n  Note: Runoff and IceDischarge stored in m³/month (raw).")
    print(f"        Convert to m³/s by dividing by {SECONDS_PER_MONTH:.0f} for plotting/analysis.")
    print(f"        SubmarineDischarge is already in m/day (Q converted internally).")


if __name__ == "__main__":
    main()
    print("\n=== All processing completed ===")

=== Processing Discharge Data ===

=== Processing Temperature Data ===
  Processed Helheim Gletsjer: 11 years of data
  Processed Storebjoern: 11 years of data
  Processed Daugaard-Jensen Gletsjer: 11 years of data
  Processed Waltershausen Gletsjer: 11 years of data

  Total data collected: 44 yearly averages

=== Calculating Submarine Melt Rates ===

=== Submarine Melt Rate Summary (m/day) ===
  NE: Mean=1.73, Min=1.28, Max=2.57
  SE: Mean=7.19, Min=4.94, Max=10.23

  Individual glacier data saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/tables/individual_glaciers_summer.csv
  Regional means data saved: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/tables/regional_means_summer.csv

DATA SUMMARY
  Total records: 44
  Glaciers: 4
  Years: 2010-2020
  Columns: ['Glacier', 'Region', 'Year', 'Runoff', 'IceDischarge', 'Temp_Avg', 'Temp_Min', 'Temp_Max', 'TF', 'SubmarineDischarge', 'Temp_Data_Points']

  Note: Runoff and IceDischarge stored in m³/month (raw).
   

In [15]:
# =============================================================================
# Section 5, Code 2: Statistical Correlation Analysis
# =============================================================================
# Pearson correlation tests at individual glacier and regional levels.
# Reads pre-computed data from Section 5 Code 1 output CSVs.
#
# Key correlations:
#   - Ocean Temperature vs Runoff
#   - Ocean Temperature vs Ice Discharge
#   - Submarine Melt Rate vs Ice Discharge (primary analytical test)
#
# All correlations reported to 3 decimal places (r and p-values).
# =============================================================================
import pandas as pd
from scipy.stats import pearsonr

# 1. Individual glacier analysis
print("=" * 60)
print("INDIVIDUAL GLACIER CORRELATION ANALYSIS")
print("=" * 60)

df = pd.read_csv('/Users/kyle/Documents/dissertation/Karlsson et al., 2023/tables/individual_glaciers_summer.csv')

glacier_results = []
for glacier in df['Glacier'].unique():
    subset = df[df['Glacier'] == glacier]
    
    # Temperature correlations
    r_corr, r_p = pearsonr(subset['Temp_Avg'], subset['Runoff'])
    i_corr, i_p = pearsonr(subset['Temp_Avg'], subset['IceDischarge'])
    
    # Submarine Melt Rate vs Ice Discharge (primary test)
    s_corr, s_p = pearsonr(subset['SubmarineDischarge'], subset['IceDischarge'])
    
    glacier_results.append({
        'Glacier': glacier,
        'Region': subset['Region'].iloc[0],
        'Temp vs Runoff (r)': f"{r_corr:.3f} (p={r_p:.3f})",
        'Temp vs IceDischarge (r)': f"{i_corr:.3f} (p={i_p:.3f})",
        'SubDischarge vs IceDischarge (r)': f"{s_corr:.3f} (p={s_p:.3f})"
    })

print(pd.DataFrame(glacier_results).to_markdown(index=False))

# 2. Regional analysis
print(f"\n{'=' * 60}")
print("REGIONAL CORRELATION ANALYSIS")
print("=" * 60)

df_reg = pd.read_csv('/Users/kyle/Documents/dissertation/Karlsson et al., 2023/tables/regional_means_summer.csv')

region_results = []
for region in df_reg['Region'].unique():
    subset = df_reg[df_reg['Region'] == region]
    
    # Temperature correlations
    r_corr, r_p = pearsonr(subset['Temp_Avg'], subset['Runoff'])
    i_corr, i_p = pearsonr(subset['Temp_Avg'], subset['IceDischarge'])
    
    # Submarine Melt Rate vs Ice Discharge (primary test)
    s_corr, s_p = pearsonr(subset['SubmarineDischarge'], subset['IceDischarge'])
    
    region_results.append({
        'Region': region,
        'Temp vs Runoff (r)': f"{r_corr:.3f} (p={r_p:.3f})",
        'Temp vs IceDischarge (r)': f"{i_corr:.3f} (p={i_p:.3f})",
        'SubDischarge vs IceDischarge (r)': f"{s_corr:.3f} (p={s_p:.3f})"
    })

print(pd.DataFrame(region_results).to_markdown(index=False))

INDIVIDUAL GLACIER CORRELATION ANALYSIS
| Glacier                  | Region   | Temp vs Runoff (r)   | Temp vs IceDischarge (r)   | SubDischarge vs IceDischarge (r)   |
|:-------------------------|:---------|:---------------------|:---------------------------|:-----------------------------------|
| Helheim Gletsjer         | SE       | 0.288 (p=0.390)      | 0.064 (p=0.851)            | -0.024 (p=0.944)                   |
| Storebjoern              | SE       | 0.605 (p=0.048)      | -0.423 (p=0.195)           | -0.406 (p=0.215)                   |
| Daugaard-Jensen Gletsjer | NE       | -0.053 (p=0.877)     | -0.309 (p=0.356)           | -0.447 (p=0.168)                   |
| Waltershausen Gletsjer   | NE       | -0.131 (p=0.701)     | 0.163 (p=0.632)            | 0.106 (p=0.757)                    |

REGIONAL CORRELATION ANALYSIS
| Region   | Temp vs Runoff (r)   | Temp vs IceDischarge (r)   | SubDischarge vs IceDischarge (r)   |
|:---------|:---------------------|:-----------------

In [16]:
# =============================================================================
# Section 5, Code 3: Correlation Scatter Plots
# =============================================================================
# Three scatter plots with regression lines:
#   (1) Runoff (m³/s) vs Potential Temperature (°C)
#   (2) Ice Discharge (m³/s) vs Potential Temperature (°C)
#   (3) Submarine Melt Rate (m/day) vs Ice Discharge (m³/s)  ← primary figure
#
# All hydrological variables converted from m³/month to m³/s at plot time.
# Conversion: m³/s = m³/month ÷ (30.44 × 86400)
# =============================================================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
from scipy.stats import pearsonr
import numpy as np
import os

# Set style
sns.set(style="whitegrid", palette="pastel", font_scale=1.2)
plt.rcParams['figure.facecolor'] = 'white'

# Configuration
output_folder = "/Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots"
os.makedirs(output_folder, exist_ok=True)

# Unit conversion constant: m³/month → m³/s
SECONDS_PER_MONTH = 30.44 * 86400

# Color scheme (consistent with other sections)
REGION_COLORS = {'NE': '#d62728',  # Dark red
                 'SE': '#1f77b4'}  # Dark blue

# Custom legend (shared across all plots)
legend_elements = [
    Line2D([0], [0], marker='o', color='w', label='Northeast (NE)',
           markerfacecolor=REGION_COLORS['NE'], markersize=10),
    Line2D([0], [0], marker='o', color='w', label='Southeast (SE)',
           markerfacecolor=REGION_COLORS['SE'], markersize=10)
]

# Load regional means data
df = pd.read_csv('/Users/kyle/Documents/dissertation/Karlsson et al., 2023/tables/regional_means_summer.csv')

# Pre-compute converted columns for plotting (m³/month → m³/s)
df['Runoff_m3s'] = df['Runoff'] / SECONDS_PER_MONTH
df['IceDischarge_m3s'] = df['IceDischarge'] / SECONDS_PER_MONTH


# =====================================================================
# Figure 1: Runoff (m³/s) vs Potential Temperature (°C)
# =====================================================================

fig1, ax1 = plt.subplots(figsize=(9, 6))

for region, color in REGION_COLORS.items():
    subset = df[df['Region'] == region]
    sns.regplot(x='Temp_Avg', y='Runoff_m3s', data=subset, 
                ax=ax1, color=color, scatter_kws={'s': 100, 'alpha': 0.7},
                line_kws={'color': color, 'linestyle': '--'}, label=region)
    
    # Correlation text (3 decimal places)
    r, p = pearsonr(subset['Temp_Avg'], subset['Runoff_m3s'])
    ax1.text(0.05, 0.9 - 0.05 * list(REGION_COLORS.keys()).index(region), 
             f"{region}: r = {r:.3f} (p = {p:.3f})",
             transform=ax1.transAxes, color=color, fontsize=12)

ax1.set_title('Mean Summer (JJA) Runoff vs Mean Summer (JJA) Potential Temperature', 
              fontsize=16, pad=15, weight='bold')
ax1.set_xlabel('Temperature (°C)', fontsize=14)
ax1.set_ylabel('Runoff (m³/s)', fontsize=14)
ax1.grid(True, linestyle=':', alpha=0.7)

fig1.legend(handles=legend_elements, loc='upper center', 
           bbox_to_anchor=(0.5, 0.05), ncol=2, fontsize=12)

plt.tight_layout()
plt.subplots_adjust(bottom=0.15)

runoff_output_path = os.path.join(output_folder, "runoff_temperature_correlation.png")
plt.savefig(runoff_output_path, dpi=300, bbox_inches='tight')
plt.close(fig1)


# =====================================================================
# Figure 2: Ice Discharge (m³/s) vs Potential Temperature (°C)
# =====================================================================

fig2, ax2 = plt.subplots(figsize=(9, 6))

for region, color in REGION_COLORS.items():
    subset = df[df['Region'] == region]
    sns.regplot(x='Temp_Avg', y='IceDischarge_m3s', data=subset, 
                ax=ax2, color=color, scatter_kws={'s': 100, 'alpha': 0.7},
                line_kws={'color': color, 'linestyle': '--'}, label=region)
    
    # Correlation text (3 decimal places)
    r, p = pearsonr(subset['Temp_Avg'], subset['IceDischarge_m3s'])
    ax2.text(0.05, 0.9 - 0.05 * list(REGION_COLORS.keys()).index(region), 
             f"{region}: r = {r:.3f} (p = {p:.3f})",
             transform=ax2.transAxes, color=color, fontsize=12)

ax2.set_title('Mean Summer (JJA) Ice Discharge vs Mean Summer (JJA) Potential Temperature', 
              fontsize=16, pad=15, weight='bold')
ax2.set_xlabel('Temperature (°C)', fontsize=14)
ax2.set_ylabel('Ice Discharge (m³/s)', fontsize=14)
ax2.grid(True, linestyle=':', alpha=0.7)

fig2.legend(handles=legend_elements, loc='upper center', 
           bbox_to_anchor=(0.5, 0.05), ncol=2, fontsize=12)

plt.tight_layout()
plt.subplots_adjust(bottom=0.15)

discharge_output_path = os.path.join(output_folder, "icedischarge_temperature_correlation.png")
plt.savefig(discharge_output_path, dpi=300, bbox_inches='tight')
plt.close(fig2)


# =====================================================================
# Figure 3: Submarine Melt Rate (m/day) vs Ice Discharge (m³/s)
# This is the study's PRIMARY analytical figure.
# =====================================================================

fig3, ax3 = plt.subplots(figsize=(10, 7))

for region, color in REGION_COLORS.items():
    subset = df[df['Region'] == region].copy()
    
    # Convert Ice Discharge to m³/s for y-axis
    subset['IceDischarge_m3s'] = subset['IceDischarge'] / SECONDS_PER_MONTH
    
    sns.regplot(x='SubmarineDischarge', y='IceDischarge_m3s', data=subset, 
                ax=ax3, color=color, scatter_kws={'s': 100, 'alpha': 0.7},
                line_kws={'color': color, 'linestyle': '--', 'linewidth': 2}, 
                label=region)
    
    # Correlation text (correlation is scale-invariant, same r whether m³/month or m³/s)
    r, p = pearsonr(subset['SubmarineDischarge'], subset['IceDischarge'])
    ax3.text(0.05, 0.9 - 0.05 * list(REGION_COLORS.keys()).index(region), 
             f"{region}: r = {r:.3f} (p = {p:.3f})",
             transform=ax3.transAxes, color=color, fontsize=13, weight='bold')

ax3.set_title('Mean Summer (JJA) Submarine Melt Rate vs Ice Discharge', 
              fontsize=17, pad=15, weight='bold')
ax3.set_xlabel(r'Submarine Melt Rate (m/day) = $0.142 \times Q^{0.31} \times TF^{1.19}$', 
               fontsize=14, weight='bold')
ax3.set_ylabel('Ice Discharge (m³/s)', fontsize=14, weight='bold')
ax3.grid(True, linestyle=':', alpha=0.7)

fig3.legend(handles=legend_elements, loc='upper center', 
           bbox_to_anchor=(0.5, 0.02), ncol=2, fontsize=12)

plt.tight_layout()
plt.subplots_adjust(bottom=0.15)

submarine_output_path = os.path.join(output_folder, "submarine_icedischarge_correlation.png")
plt.savefig(submarine_output_path, dpi=300, bbox_inches='tight')
plt.close(fig3)

print(f"Runoff correlation plot saved to: {runoff_output_path}")
print(f"Ice discharge correlation plot saved to: {discharge_output_path}")
print(f"Submarine melt rate correlation plot saved to: {submarine_output_path}")

Runoff correlation plot saved to: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/runoff_temperature_correlation.png
Ice discharge correlation plot saved to: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/icedischarge_temperature_correlation.png
Submarine melt rate correlation plot saved to: /Users/kyle/Documents/dissertation/Karlsson et al., 2023/plots/submarine_icedischarge_correlation.png


## Bibliography

Copernicus Climate Data Store (CDS) (2021) ORAS5 Global Ocean Reanalysis Monthly Data from 1958 to Present. *Copernicus Climate Change Service (C3S)*. DOI: https://doi.org/10.24381/cds.67e8eeb7

Google Earth (2025) *Maps showing the location of Helheim Gletsjer, Storebjoern, Daugaard-Jensen Gletsjer, and Waltershausen Gletsjer.* Available at: https://earth.google.com/web/@73.8999997,-24.41666671,248.07216786a,950.49443682d,35y,-0h,0t,0r/data=CgRCAggBOgMKATBCAggBSg0I____________ARAA (Accessed on 05-07-2025)

Google Maps API (2025) Google Terrain tiles Retrieved from https://mapsplatform.google.com/ (Accessed on 05-07-2025)

Jackson, R. H., Nash, J. D., Kienholz, C., Sutherland, D. A., Amundson, J. M., Motyka, R. J., Winters, D., Skyllingstad, E., Pettit, E. C. (2020). Meltwater Intrusions Reveal Mechanisms for Rapid Submarine Melt at a Tidewater Glacier. *Geophysical Research Letters*, 47, e2019GL085335. DOI: https://doi.org/10.1029/2019GL085335

Jakobsson, M., Mohammad, R., Karlsson, M. et al. (2024) The International Bathymetric Chart of the Arctic Ocean Version 5.0. *Scientific Data*, 11, 1420. DOI: https://doi.org/10.1038/s41597-024-04278-w

Karlsson, N. B., Mankoff, K. D., Solgaard, A. M., Larsen, S. H., How, P. R., Fausto, R. S., Sørensen, L. S. (2023) A Data Set of Monthly Freshwater Fluxes from the Greenland Ice Sheet’s Marine-terminating Glaciers on a Glacier–Basin Scale 2010–2020. *Geological Survey of Denmark and Greenland*, 53, 8338. DOI: https://doi.org/10.34194/geusb.v53.8338

Seale, A., Christoffersen, P., Mugford, R. I., O’Leary, M. (2011) Ocean Forcing of the Greenland Ice Sheet: Calving Fronts and Patterns of Retreat Identified by Automatic Satellite Monitoring of Eastern Outlet Glaciers. *Journal of Geophysical Research*, 116, F03015. DOI: https://doi.org/10.1029/2010JF001847

Slater, D.A. and Straneo, F. (2022) 'Submarine melting of glaciers in Greenland amplified by atmospheric warming', *Nature Geoscience*, 15(10), pp. 794–799. doi:10.1038/s41561-022-01035-9.

Straneo, F., Hamilton, G.S., Sutherland, D.A., Stearns, L.A., Davidson, F., Hammill, M.O., Stenson, G.B. and Rosing-Asvid, A. (2010) 'Rapid circulation of warm subtropical waters in a major glacial fjord in East Greenland', *Nature Geoscience*, 3(3), pp. 182–186. doi:10.1038/ngeo764.